![](https://i.imgur.com/N0zUCi0.png)

# Prompt Engineering with LLMs on Real-World Tasks

This notebook provides a **hands-on, end-to-end walkthrough of prompt engineering techniques** applied to practical, real-world business and technical use cases using Large Language Models (LLMs).

Rather than focusing on abstract theory, the notebook demonstrates **how different prompting patterns influence model behavior**, output quality, structure, and reasoning across a variety of tasks commonly encountered in enterprise and product workflows.

---

## What You Will Learn

By working through this notebook, you will learn how to:

- Design clear and structured prompts for different problem types  
- Apply zero-shot, few-shot, and template-based prompting patterns  
- Extract structured information from unstructured text  
- Generate professional, context-aware responses using LLMs  
- Use LLMs for analysis, summarization, content generation, classification, and reasoning  
- Compare basic prompting with advanced reasoning techniques such as Chain-of-Thought and Self-Consistency  
- Produce structured outputs using schemas (Pydantic + LangChain)

---

## Notebook Structure and Flow

The notebook is organized into the following stages:

### 1. Environment Setup
- Install and load required dependencies  
- Configure API access for LLM providers  
- Create reusable helper functions for interacting with chat-based LLMs  

---

### 2. Real-World Prompt Engineering Tasks

Each task demonstrates a specific application of prompt engineering, gradually increasing in complexity.

#### Task 1: Review Analysis, Information Extraction, and Response Generation
- Analyze customer reviews  
- Extract key issues and sentiment  
- Generate professional, customer-friendly responses  

#### Task 2: Paper Analysis and Summarization
- Summarize long research papers or articles  
- Extract key insights, contributions, and conclusions  
- Reduce technical content into concise summaries  

#### Task 3: Marketing Advert Generation
- Generate marketing copy using product specifications  
- Adapt tone and messaging for promotional use cases  
- Demonstrate controlled creativity through prompting  

#### Task 4: IT Support Resolution Using Output Templates
- Generate structured technical support responses  
- Use prompt templates to ensure consistency and completeness  
- Simulate automated helpdesk workflows  

#### Task 5: IT Support Resolution with Structured Outputs
- Extend template-based prompting using structured output schemas  
- Enforce response format using LangChain and Pydantic  
- Compare structured versus unstructured LLM outputs  

#### Task 6: LLM as a Classifier Using Few-Shot Prompting
- Use examples directly inside prompts to guide classification behavior  
- Perform text classification without training a traditional ML model  
- Demonstrate how few-shot prompting improves consistency and accuracy  

#### Task 7: Advanced Prompting Patterns for Reasoning

This section focuses on improving reasoning quality, not just final answers.

Covered techniques include:

- Zero-shot Chain-of-Thought  
- Few-shot Chain-of-Thought  
- Self-Consistency Sampling  

You will see:
- How reasoning traces improve complex problem solving  
- When each technique is useful  
- A comparison of outputs across different reasoning strategies  



## Install Dependencies

This section installs the required Python libraries needed to interact with LLM models.

We install:

- langchain → To interface with various LLMs
- langchain-openai → To interact with OpenAI GPT LLMs
- langchain-groq → To interact with Groq-hosted open LLMs like GPT-OSS-20B

LangChain provides a convenient interface to connect with multiple LLM providers using a unified framework.

Installing these libraries ensures we can easily call and switch between models.

In [ ]:
!pip install -q langchain==1.2.8
!pip install -q langchain-openai==1.1.7
!pip install -q langchain-groq==1.1.2

## Load Necessary Dependencies

In [ ]:
import os # Import the os module for interacting with the operating system
from getpass import getpass # Import getpass for securely prompting for API keys

from langchain_openai import ChatOpenAI # Import ChatOpenAI for interacting with OpenAI chat models
from langchain_groq import ChatGroq # Import ChatGroq for interacting with Groq chat models
from langchain_core.messages import HumanMessage # Import HumanMessage for constructing chat messages

---

## Setup Environment Variables

Set up your API keys securely. You'll need to obtain:

- **OpenAI API Key**: Get from [OpenAI Platform](https://platform.openai.com/api-keys)
- **Groq API Key**: Get from [Groq Console](https://console.groq.com/keys)

In [ ]:
import os
from getpass import getpass

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Set up Groq API key
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")

Enter your OpenAI API Key: ··········
Enter your Groq API Key: ··········


## Create OpenAI and Groq Chat Completion Access Function

This section defines functions that allow us to interact with LLM models easily.

The function will:

- Accept prompts
- Send them to selected model
- Receive response
- Return output

This abstraction makes it easy to:

- Switch models
- Reuse code
- Compare responses

In [ ]:
def get_completion(prompt, model="gpt-4.1-mini"):
    """
    Generates a completion from the specified LLM model based on the given prompt.

    Args:
        prompt (str): The input text prompt for the LLM.
        model (str, optional): The name of the LLM model to use.
                               Defaults to "gpt-4.1-mini".
                               Supported models: "gpt-4.1-mini", "openai/gpt-oss-20b".

    Returns:
        str: The generated response content from the LLM.
             Returns an error message if the model is not configured.
    """
    # Create a list of messages, starting with a human message containing the prompt
    messages = [HumanMessage(content=prompt)]

    # Check which model is requested
    if model == "gpt-4.1-mini":
        # Initialize ChatOpenAI with the specified model and temperature (0 for deterministic output)
        llm = ChatOpenAI(model_name=model, temperature=0)
        # Invoke the LLM with the messages and get the response
        response = llm.invoke(messages)
        # Return the content of the response
        return response.content

    elif model == "openai/gpt-oss-20b":
        # Initialize ChatGroq with the Groq API key, specified model, and temperature
        llm = ChatGroq(model_name=model, temperature=0)
        # Invoke the LLM with the messages and get the response
        response = llm.invoke(messages)
        # Return the content of the response
        return response.content
    else:
        # Return an error message if the requested model is not configured
        return "LLM not configured! Please configure logic for specific model in get_completion()"

## Testing Model Access

Before solving real tasks, we test whether both models are working correctly.

This ensures:

- API keys are correct
- Models are responding
- Integration is successful

This step prevents errors later.

In [ ]:
# Import display and Markdown from IPython.display for rich output formatting in notebooks
from IPython.display import display, Markdown

In [ ]:
# Call the get_completion function to test the 'gpt-4.1-mini' model
response = get_completion(prompt='Explain Generative AI in 2 bullet points',
                          model='gpt-4.1-mini')
# Display the response in Markdown format
display(Markdown(response))

- Generative AI refers to artificial intelligence systems designed to create new content—such as text, images, music, or code—by learning patterns from existing data.
- It uses models like neural networks to generate outputs that mimic human creativity, enabling applications like chatbots, image synthesis, and automated content creation.

In [ ]:
# Call the get_completion function to test the 'openai/gpt-oss-20b' model
response = get_completion(prompt='Explain Generative AI in 2 bullet points',
                          model='openai/gpt-oss-20b')
# Display the response in Markdown format
display(Markdown(response))

- **Content creation engine** – Generative AI models (e.g., GPT‑4, DALL‑E, Stable Diffusion) learn patterns from vast datasets and then synthesize brand‑new text, images, audio, or code that mimics the style and structure of the training data.  
- **Probabilistic sequence generation** – These models predict the next token (word, pixel, note, etc.) in a sequence by estimating probabilities, allowing them to produce coherent, context‑aware outputs that can be fine‑tuned for specific tasks or creative applications.

## Task 1: Review Analysis, Information Extraction & Response Generation

For each review get LLMs to do the following:


- Summarize the following reviews below, delimited by triple
backticks.
- For each review, the summary should be at most 3 lines.
- Highlight both the positives and negatives for each review
- Display the overall sentiment for each review (positive, negative, neutral)
- Display a list of 3 - 5 emotions expressed by the customer
- If the sentiment is positive or neutral write an email and thank them for the review
- If the sentiment is negative apologize and write an email with an appropriate response

In [ ]:
# Define a list of customer reviews for product analysis
reviews = [
    # Review 1: Bluetooth speaker
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    # Review 2: Gaming keyboard
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    # Review 3: Wireless earbuds
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    # Review 4: Tablet stand
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    # Review 5: Kitchen blender
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [ ]:
from tqdm import tqdm

# Initialize empty lists to store responses from GPT 4.1 mini and GPT OSS models
gpt4_responses = []
gptoss_responses = []

# Iterate through each review in the 'reviews' list
for review in tqdm(reviews):
    # Construct a detailed prompt for the LLM to act as a product review analyst
    prompt = f"""
Act as a product review analyst.
Your objective is to perform the following tasks:

- Summarize the following review below, delimited by triple backticks in at most 3 lines.
- Highlight both the positives and negatives for the review separately if any
- Display the overall sentiment for each review (positive, negative OR neutral)
- Display a list of 3 - 5 emotions expressed by the customer
- If the sentiment is positive or neutral write an email and thank them for the review
- If the sentiment is negative apologize and write an email with an appropriate response

```{review}```
"""
    # Get completion from the 'gpt-4.1-mini' model (OpenAI) and append to its response list
    response = get_completion(prompt,
                              model='gpt-4.1-mini')
    gpt4_responses.append(response)
    # Get completion from the 'openai/gpt-oss-20b' model (Groq) and append to its response list
    response = get_completion(prompt,
                              model='openai/gpt-oss-20b')
    gptoss_responses.append(response)

100%|██████████| 5/5 [00:34<00:00,  6.96s/it]


In [ ]:
# Iterate through each original review and its corresponding OpenAI response
for review, response in zip(reviews, gpt4_responses):
    print('Review:')
    # Display the original review in Markdown format
    display(Markdown(review))
    print()
    print('LLM Response (GPT-4.1-mini):')
    # Display the OpenAI model's response in Markdown format
    display(Markdown(response))
    print('------')
    print('\n')

Review:



    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    


LLM Response (GPT-4.1-mini):


**Summary:**  
The customer is very pleased with the Bluetooth speaker, praising its clear sound quality, waterproof feature, compact size with powerful volume, affordable price, and timely shipping.

**Positives:**  
- Impressive sound quality with balanced bass  
- Waterproof and durable  
- Compact yet powerful volume  
- Affordable price  
- Early and secure shipping  

**Negatives:**  
- None mentioned  

**Overall Sentiment:**  
Positive

**Emotions Expressed:**  
- Satisfaction  
- Excitement  
- Trust  
- Appreciation  
- Happiness

---

**Email Response:**

Subject: Thank You for Your Wonderful Review!

Dear Customer,

Thank you so much for taking the time to share your positive experience with our Bluetooth speaker. We're thrilled to hear that it met your expectations in sound quality, durability, and value, and that it arrived promptly and safely. Your feedback truly motivates us to continue delivering great products and service.

Enjoy your beach outings with your new speaker!

Best regards,  
[Your Company Name] Customer Support Team

------


Review:


...

------


Review:



    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    


LLM Response (GPT-4.1-mini):


**Summary:**  
The customer is disappointed with the kitchen blender, citing poor performance with tough foods, excessive noise, and a faulty easy-clean feature. Additionally, the product arrived late, leading to dissatisfaction with the overall purchase.

**Positives:**  
- None mentioned.

**Negatives:**  
- Struggles with tough foods  
- Very noisy operation  
- Easy-clean feature ineffective  
- Late delivery  
- Disappointment with brand quality

**Overall Sentiment:** Negative

**Emotions Expressed:**  
- Frustration  
- Disappointment  
- Annoyance  
- Regret  
- Distrust

---

**Response Email:**

Subject: We Apologize for Your Unsatisfactory Experience with Our Blender

Dear [Customer Name],

Thank you for taking the time to share your feedback. We sincerely apologize for the inconvenience and frustration caused by the blender’s performance, noise level, and the ineffective easy-clean feature. We also regret the delay in delivery, which added to your dissatisfaction.

Your experience does not reflect the quality we strive to provide, and we would like to make this right. Please contact our customer service team at [contact information], and we will be happy to assist you with a replacement, refund, or any other solution that suits you best.

We appreciate your patience and hope to restore your trust in our brand.

Best regards,  
[Your Name]  
Customer Service Team

------




In [ ]:
# Iterate through each original review and its corresponding GPT OSS LLM response
for review, response, in zip(reviews, gptoss_responses):
    print('Review:')
    # Display the original review in Markdown format
    display(Markdown(review))
    print()
    print('LLM Response:')
    # Display the GPT OSS model's response in Markdown format
    display(Markdown(response))
    print('------')
    print('\n')

Review:



    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    


LLM Response:


**Summary (≤3 lines)**  
The reviewer praises the Bluetooth speaker’s clear sound, bass, waterproof feature, compact size, and great value, noting fast, secure shipping. No drawbacks were mentioned.  

**Positives**  
- Excellent sound quality and bass  
- Waterproof and durable  
- Compact yet powerful volume  
- Great price/value  
- Fast, secure shipping  

**Negatives**  
- None reported  

**Overall Sentiment**  
Positive  

**Emotions Expressed**  
1. Satisfaction  
2. Excitement  
3. Trust  
4. Appreciation  
5. Confidence  

---

**Thank‑You Email**

Subject: Thank You for Your Fantastic Review!

Dear [Customer Name],

Thank you so much for taking the time to share your experience with our Bluetooth speaker. We’re thrilled to hear that the sound quality, waterproof design, and value met—and even exceeded—your expectations. It’s also great to know that our shipping process delivered the product quickly and safely.

Your feedback helps us continue to provide high‑quality products and service. If you have any further questions or need assistance, please don’t hesitate to reach out.

Enjoy your beach outings and the music that comes with them!

Warm regards,

[Your Name]  
Customer Experience Team  
[Company Name]  
[Contact Information]

------


Review:


...

------


Review:



    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    


LLM Response:


**Summary (≤ 3 lines)**  
The blender fails to process tougher foods, is very noisy, and its easy‑clean feature doesn’t work, causing food to get stuck. It arrived late and the customer feels it isn’t worth the price. Overall, the experience was disappointing.

**Positives**  
- None mentioned.

**Negatives**  
- Struggles with tough foods (e.g., cooked vegetables).  
- Extremely noisy operation.  
- Easy‑clean feature ineffective; food sticks under blades.  
- Brand quality expectations unmet.  
- Late delivery (3 days late).  
- Not worth the expense.

**Overall Sentiment**  
Negative

**Emotions Expressed**  
1. Frustration  
2. Disappointment  
3. Annoyance  
4. Anger  
5. Regret  

---

### Apology & Response Email

Subject: We’re sorry you’re disappointed with your new blender – let’s fix this

Dear [Customer Name],

Thank you for taking the time to share your experience with our blender. I’m truly sorry to hear that it hasn’t lived up to your expectations and that you’ve encountered several issues—from performance and noise to the easy‑clean feature and delivery delays.

Your feedback is invaluable, and we’re committed to making things right. Here’s what we can do for you:

1. **Replacement or Refund** – We can ship a brand‑new unit at no extra cost, or if you prefer, we’ll issue a full refund.  
2. **Extended Warranty** – If you choose a replacement, we’ll add a 12‑month extended warranty at no charge.  
3. **Customer Support** – Our technical team is ready to walk you through troubleshooting steps or help you set up the new unit.  
4. **Compensation** – As a gesture of goodwill, we’d like to offer you a 15 % discount on any future purchase.

Please let us know which option works best for you, and we’ll arrange the next steps immediately. You can reply to this email or call our dedicated support line at 1‑800‑BLENDER‑HELP.

Again, I apologize for the inconvenience and appreciate your patience. We value your business and hope to restore your confidence in our products.

Warm regards,

[Your Name]  
Customer Experience Team  
[Company Name]  
[Phone] | [Email] | [Website]

------




## Task 2: Paper Analysis and Summarization

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise summary of maximum 10 lines

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper.

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare

In [ ]:
# Define a multi-line string variable to hold a research paper abstract
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

In [ ]:
# Define the prompt for generating a short, concise summary of the research paper abstract.
# The prompt instructs the AI to act as an Artificial Intelligence Expert.
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise summary of maximum 10 lines

Abstract:
```{paper_abstract}```
"""

In [ ]:
# Print the constructed prompt to review its content before sending it to the LLM.
print(prompt)


Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise summary of maximum 10 lines

Abstract:
```
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate a short summary of the paper abstract.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated summary in Markdown format.
display(Markdown(response))

This study systematically reviews ethical discussions on generative AI in healthcare across text, image, and structured data modalities. It identifies gaps in current debates, especially beyond guideline adherence, and proposes an ethics checklist to improve transparency and trust in AI applications. For text-based AI, particularly large language models like GPT, ethical concerns such as non-maleficence, equity, and privacy are widely discussed, with some works proposing AI-driven solutions to issues like data confidentiality and misinformation. In contrast, image and structured data applications focus mainly on synthetic data generation to address privacy and bias, but often lack in-depth ethical analysis on autonomy, integrity, or morality. The checklist aims to integrate ethical assessment into research and real-world AI deployment, fostering responsible innovation in healthcare.

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate a short summary of the paper abstract.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated summary in Markdown format.
display(Markdown(response))

**Summary (≤10 lines)**  

1. The rise of generative AI (e.g., ChatGPT, LLMs, GANs) in healthcare raises ethical concerns that current guidelines and regulations have not fully addressed.  
2. A systematic scoping review of 193 relevant studies (2013‑2023) revealed that most discussions focus on text‑based AI, with fewer on image or structured data.  
3. Ethical issues identified include non‑maleficence, equity, privacy, human‑AI interaction, and the potential authorship rights of AI.  
4. While many papers highlight problems caused by generative AI, only a minority propose AI‑driven solutions (e.g., synthetic data to protect privacy, bias mitigation).  
5. Image and structured‑data studies largely concentrate on privacy protection via synthetic data, with limited discussion of autonomy, integrity, or moral implications.  
6. The review uncovers gaps: uneven coverage of ethical principles, lack of balanced cause‑and‑solution analyses, and insufficient transparency in AI‑generated content.  
7. To bridge these gaps, the authors propose an ethics checklist for comprehensive assessment and transparent documentation of generative AI projects.  
8. The checklist can be integrated into peer review and product disclosure, fostering user trust and responsible deployment of generative AI in healthcare.

In [ ]:
# Define the prompt for generating an executive summary for a healthcare company.
# The prompt instructs the AI to act as an Artificial Intelligence Expert and include
# bullet points for pros and cons of ethics in Generative AI.
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper.

Abstract:
```{paper_abstract}```
"""

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate the executive summary for a healthcare company.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated executive summary in Markdown format.
display(Markdown(response))

**Executive Summary: Ethical Considerations of Generative AI in Healthcare**

Generative Artificial Intelligence (AI), including technologies like ChatGPT and other Large Language Models (LLMs), is rapidly transforming healthcare by enabling new applications in text, image, and structured data processing. However, the ethical implications of these technologies remain complex and evolving, especially in high-stakes healthcare environments. This summary distills key insights from a comprehensive review of recent research on the ethical challenges and opportunities posed by generative AI in healthcare, and proposes a practical ethics checklist to guide responsible AI development and deployment.

---

### Key Insights

- **Scope of Review:**  
  Analysis of 193 relevant articles (from 2013 to mid-2023) covering generative AI applications in healthcare text, image, and structured data.

- **Text Data Applications:**  
  - Focus on LLMs (e.g., GPT) with extensive ethical discussions around principles such as non-maleficence, equity, and privacy.  
  - Ethical challenges include data confidentiality, misinformation, and human-AI interaction.  
  - Generative AI is also used to address ethical issues by generating synthetic medical text, improving accessibility, and enhancing transparency.

- **Image and Structured Data Applications:**  
  - Primarily used for data synthesis and encryption to protect privacy.  
  - Ethical discussions are less frequent and more narrowly focused compared to text data.  
  - Efforts include reducing bias by generating synthetic data for underrepresented groups.  
  - Limited discourse on autonomy, integrity, morality, trust, or transparency in these modalities.

- **Ethics Checklist Proposal:**  
  A comprehensive tool designed to systematically assess and document ethical considerations in generative AI development, facilitating transparency and trust. This checklist can be integrated into peer review, publication processes, and product development workflows.

---

### Pros and Cons of Ethics in Generative AI (as highlighted in the research)

**Pros:**

- **Enhanced Privacy Protection:**  
  Use of synthetic data generation reduces risks related to patient confidentiality and data breaches.

- **Bias Mitigation:**  
  Generative AI can help balance datasets by creating synthetic data for underrepresented populations, promoting equity.

- **Improved Accessibility and Transparency:**  
  AI-powered tools can provide more equitable healthcare services and improve accountability in information dissemination.

- **Detection of Misinformation:**  
  Generative AI can identify and reduce health-related misinformation, supporting better clinical decision-making.

- **Facilitation of Ethical Research Practices:**  
  The ethics checklist encourages transparent documentation and comprehensive ethical assessment, fostering trust among users and stakeholders.

---

**Cons:**

- **Ethical Ambiguity and Gaps:**  
  Current guidelines and regulations are still under development, leaving unresolved ethical dilemmas.

- **Limited Discussion on Autonomy and Morality:**  
  Few studies address how generative AI impacts patient autonomy, moral considerations, or integrity of healthcare decisions.

- **Potential for New Ethical Risks:**  
  Generative AI may introduce biases or inaccuracies, especially when synthetic data is used without thorough validation.

- **Human-AI Interaction Challenges:**  
  Issues such as the rights of AI systems (e.g., as co-authors) and the dynamics of human-AI collaboration remain underexplored.

- **Transparency and Trust Deficits:**  
  Particularly in image and structured data applications, there is a lack of explicit focus on building user trust and ensuring transparent AI behavior.

---

### Implications for Healthcare Companies

- **Adopt Ethical Frameworks:**  
  Integrate comprehensive ethics checklists into AI product development and evaluation to proactively address ethical risks.

- **Focus on Transparency:**  
  Clearly communicate AI capabilities, limitations, and ethical considerations to healthcare providers and patients to build trust.

- **Prioritize Privacy and Equity:**  
  Leverage generative AI’s potential to protect patient data and reduce healthcare disparities through synthetic data and bias mitigation.

- **Engage in Ongoing Ethical Dialogue:**  
  Stay informed on evolving regulations and participate in shaping ethical standards for generative AI in healthcare.

- **Balance Innovation with Responsibility:**  
  While generative AI offers transformative benefits, careful ethical scrutiny is essential to ensure safe, fair, and trustworthy healthcare applications.

---

This executive summary underscores the dual nature of generative AI in healthcare: a powerful enabler of innovation and a source of complex ethical challenges. By adopting structured ethical assessments and fostering transparent practices, healthcare companies can harness generative AI responsibly to improve patient outcomes and maintain public trust.

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate the executive summary for a healthcare company.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated executive summary in Markdown format.
display(Markdown(response))

**Executive Summary – Generative AI in Healthcare**

*Prepared for: Executive Leadership Team, [Healthcare Company]*  
*Date: 13 Feb 2026*  

---

### 1. Why This Matters

- **Rapid Adoption**: ChatGPT‑style large language models (LLMs) and other generative AI (GenAI) tools are now embedded in clinical decision support, patient communication, and research workflows.
- **High‑stakes Environment**: Errors or ethical lapses can directly affect patient safety, data privacy, and regulatory compliance.
- **Regulatory Gap**: Current guidelines are still evolving; companies must proactively address ethical concerns to build trust and avoid liability.

---

### 2. Key Findings from the Systematic Review (Jan 2013 – Jul 2023)

| Category | Scope | Highlights |
|----------|-------|------------|
| **Total literature** | 2,859 screened | 193 articles met inclusion criteria |
| **Data modalities** | Text, Image, Structured | 41 (text), 98 (image), 58 (structured) |
| **Ethical Principles Discussed** | 10 core principles (e.g., non‑maleficence, equity, privacy) | Non‑maleficence, equity, privacy most frequent |
| **LLM‑specific issues** | 29 articles | Focus on GPT‑style models; concerns about bias, misinformation, authorship rights |
| **Synthetic data solutions** | 15 articles | Use of GANs, autoencoders, diffusion models to protect privacy, reduce bias, and improve transparency |
| **Gaps Identified** |  | • Limited discussion on autonomy, integrity, morality, trust, and transparency for image/structured data.<br>• Few studies balance “cause” and “solution” perspectives. |
| **Proposed Tool** | Ethics Checklist | Designed for peer‑review integration and product disclosure to foster user trust. |

---

### 3. Practical Implications for Our Company

| Area | Opportunity | Risk |
|------|-------------|------|
| **Data Privacy** | Generate synthetic patient records to train models without exposing PHI. | Synthetic data may still leak identifiable patterns if not properly vetted. |
| **Bias Mitigation** | Augment under‑represented groups in training sets to improve equity. | Synthetic augmentation can introduce new biases if source data are flawed. |
| **Transparency & Accountability** | Use the ethics checklist to document decision‑making and model provenance. | Checklist adoption requires cultural change and training. |
| **Regulatory Compliance** | Early ethical documentation can satisfy emerging FDA/EMA AI‑device guidelines. | Lack of clear regulations may still expose us to unforeseen penalties. |
| **Patient Trust** | Transparent disclosure of GenAI use can enhance patient confidence. | Miscommunication or over‑promising capabilities may erode trust. |

---

### 4. Pros & Cons of Ethics in Generative AI (as highlighted in the paper)

#### Pros
- **Privacy Protection** – Synthetic data generation reduces direct exposure of real patient information.  
- **Bias Reduction** – Data synthesis can balance demographic representation, improving fairness.  
- **Misinformation Detection** – GenAI can flag or correct false health claims in real time.  
- **Enhanced Transparency** – Structured documentation (checklist) clarifies model intent and limitations.  
- **Accountability** – Clear ethical frameworks help trace responsibility for decisions made by AI.  

#### Cons
- **Residual Privacy Risks** – Synthetic data may still encode sensitive patterns if not properly anonymized.  
- **Unintended Bias** – Poorly designed generative models can amplify existing disparities.  
- **Autonomy & Integrity Gaps** – Current literature rarely addresses patient autonomy or data integrity in GenAI contexts.  
- **Regulatory Uncertainty** – Guidelines are still under development; compliance may lag behind technology deployment.  
- **Ethical Oversight Burden** – Implementing comprehensive checklists demands resources and may slow innovation cycles.  

---

### 5. Recommendations

1. **Adopt the Proposed Ethics Checklist**  
   - Integrate into internal review pipelines and product documentation.  
   - Use it as a baseline for external audits and regulatory submissions.

2. **Pilot Synthetic Data Projects**  
   - Start with low‑risk, non‑clinical datasets to validate privacy safeguards.  
   - Monitor for inadvertent bias introduction.

3. **Establish an Ethics Governance Board**  
   - Include clinicians, data scientists, legal counsel, and patient advocates.  
   - Review all GenAI projects for autonomy, integrity, and moral considerations.

4. **Invest in Training & Culture**  
   - Educate staff on ethical principles and the practical use of the checklist.  
   - Foster a culture where ethical concerns are raised early, not as after‑thoughts.

5. **Engage with Regulators & Standards Bodies**  
   - Participate in shaping forthcoming AI‑device guidelines.  
   - Stay ahead of compliance requirements by aligning internal practices with emerging standards.

---

**Bottom Line:** Generative AI offers powerful tools to enhance patient care and operational efficiency, but only if deployed within a robust ethical framework. By proactively adopting the checklist and addressing the identified gaps, we can unlock GenAI’s benefits while safeguarding patient trust and regulatory compliance.

In [ ]:
# Define the prompt for generating an executive summary for a generative AI company solving healthcare problems.
# The prompt instructs the AI to act as an Artificial Intelligence Expert and include
# key points for Generative AI for text, images, and structured data-based healthcare.
prompt = f"""
Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare

Abstract:
```{paper_abstract}```
"""

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate the executive summary for a generative AI company.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated executive summary in Markdown format.
display(Markdown(response))

**Executive Summary: Leveraging Generative AI to Solve Healthcare Challenges**

As a leading generative AI company focused on healthcare, we recognize both the transformative potential and ethical complexities of deploying generative AI technologies across diverse healthcare data modalities—text, images, and structured data. Our approach is informed by a comprehensive review of current research and ethical discourse, enabling us to build trustworthy, effective AI solutions that address critical healthcare needs while upholding rigorous ethical standards.

---

### Key Insights on Generative AI in Healthcare

#### 1. **Generative AI for Text-Based Healthcare Applications**
- **Focus:** Large Language Models (LLMs) like GPT dominate text-based healthcare AI applications.
- **Ethical Considerations:** 
  - Addressed extensively across 29 studies, covering principles such as non-maleficence (avoiding harm), equity, privacy, human-AI interaction, and authorship rights.
  - Privacy and confidentiality of sensitive medical text data are paramount concerns.
- **Solutions Enabled by Generative AI:**
  - Synthetic medical text generation to protect patient confidentiality.
  - Tools to detect and reduce health misinformation.
  - Enhancing accessibility and reducing disparities in healthcare services.
  - Improving transparency and accountability in AI-generated content.
- **Balanced Perspectives:** Some research integrates both ethical challenges and AI-driven solutions, fostering responsible innovation.

#### 2. **Generative AI for Image-Based Healthcare Applications**
- **Focus:** Primarily on data synthesis and encryption using generative adversarial networks (GANs).
- **Ethical Challenges:**
  - Privacy protection through synthetic image data generation.
  - Addressing bias by augmenting datasets with under-represented subgroups.
  - Limited discussion on autonomy, integrity, or morality concerns.
- **Applications:**
  - Creating realistic synthetic medical images to enable research without compromising patient privacy.
  - Enhancing dataset diversity to improve diagnostic AI fairness.

#### 3. **Generative AI for Structured Data in Healthcare**
- **Focus:** Synthetic data generation to safeguard privacy and improve data security.
- **Ethical Considerations:**
  - Privacy and data security are the primary concerns addressed.
  - Limited discourse on trust, transparency, autonomy, or morality.
  - Some recognition of bias introduced during synthetic data creation.
- **Use Cases:**
  - Generating synthetic structured datasets for research and model training.
  - Mitigating privacy risks while maintaining data utility.

---

### Our Commitment and Strategic Approach

- **Ethics-First Development:** We integrate a comprehensive ethics checklist into our AI development lifecycle, ensuring transparent documentation and assessment of ethical considerations.
- **Multi-Modal Expertise:** Our solutions span text, image, and structured data, enabling holistic healthcare applications from clinical documentation to medical imaging and electronic health records.
- **Privacy-Preserving Innovation:** Leveraging synthetic data generation to protect patient confidentiality without compromising data quality or utility.
- **Bias Mitigation:** Actively addressing equity by augmenting datasets and refining models to serve diverse populations fairly.
- **Transparency & Trust:** Enhancing explainability and accountability in AI outputs to build user confidence and meet regulatory expectations.
- **Balanced Ethical Perspectives:** Combining identification of ethical risks with AI-driven solutions to foster responsible and impactful healthcare innovation.

---

By harnessing the power of generative AI responsibly across multiple healthcare data types, we are uniquely positioned to drive breakthroughs that improve patient outcomes, streamline clinical workflows, and uphold the highest ethical standards in healthcare technology.

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate the executive summary for a generative AI company.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated executive summary in Markdown format.
display(Markdown(response))

**Executive Summary – Generative AI for Healthcare**

*Our company is positioned to deliver AI‑driven solutions that transform patient care, research, and operational efficiency while upholding the highest ethical standards. The following points distill the latest research landscape and outline how our platform addresses the most pressing challenges across text, image, and structured‑data modalities.*

---

### 1. Market Opportunity & Ethical Landscape
- **Rapid adoption of LLMs (ChatGPT‑style) and image‑generation models (GANs, diffusion) in clinical settings** has accelerated, but **ethical concerns—privacy, bias, accountability, and transparency—remain under‑addressed.**
- **Regulatory frameworks are still evolving**; companies must go beyond compliance and embed ethics into product design.
- **Existing literature (2859 papers, 193 in‑depth analyses) shows a clear gap**: many studies identify problems but few provide actionable solutions or balanced viewpoints.

---

### 2. Our Solution Framework
- **Integrated Ethics Checklist** – a modular, peer‑review‑ready tool that documents every ethical consideration (non‑maleficence, equity, privacy, autonomy, integrity, trust, transparency, etc.) during model development, deployment, and post‑market monitoring.
- **Transparent Documentation Portal** – real‑time audit trails and explainability dashboards that allow clinicians, regulators, and patients to verify model decisions.
- **Trust‑Building Features** – automated bias audits, synthetic data generation for under‑represented groups, and privacy‑preserving data synthesis (federated learning, differential privacy).

---

### 3. Key Findings by Data Modality

| Modality | Core Ethical Issues | Generative AI Solutions | Our Product Edge |
|----------|---------------------|------------------------|------------------|
| **Text (LLMs)** | • Privacy & confidentiality of clinical notes<br>• Bias & misinformation<br>• Accountability & authorship<br>• Human‑AI interaction | • Synthetic de‑identified text generation<br>• Misinformation detection & correction<br>• Transparent provenance tracking | • Domain‑specific LLMs fine‑tuned on de‑identified EHRs<br>• Built‑in audit logs for every generated token<br>• Consent‑aware data pipelines |
| **Images (GANs, Diffusion)** | • Patient privacy in imaging<br>• Bias in training datasets (under‑represented anatomies)<br>• Data security & encryption | • Synthetic image generation for training & augmentation<br>• Privacy‑preserving image encryption<br>• Bias‑mitigation via controlled synthesis | • On‑premise synthetic image engine with differential privacy guarantees<br>• Real‑time bias scoring for each generated image |
| **Structured Data (EHR tables, claims)** | • Privacy & re‑identification risk<br>• Bias in demographic representation<br>• Lack of transparency in synthetic data generation | • GAN‑based synthetic tabular data<br>• Differentially private data synthesis<br>• Bias‑adjusted sampling for minority groups | • End‑to‑end synthetic data platform with audit trails<br>• Automated compliance checks against HIPAA & GDPR |

---

### 4. Strategic Advantages for a Generative‑AI Healthcare Company

1. **Ethics‑First Design** – Our checklist is embedded in every stage, ensuring that ethical gaps identified in the literature are proactively closed.
2. **Regulatory Readiness** – The documentation portal aligns with emerging FDA, EMA, and HIPAA guidelines, reducing time‑to‑market.
3. **Trust & Adoption** – Transparent provenance and bias metrics build clinician confidence and patient trust.
4. **Scalable Data Synthesis** – Synthetic data reduces the need for costly, privacy‑restricted real data while maintaining model performance.
5. **Balanced Problem‑Solution Approach** – Unlike many studies that only highlight issues, our platform simultaneously identifies risks and delivers concrete mitigations.

---

### 5. Call to Action

- **Partner with us** to integrate our ethics‑centric generative AI platform into your clinical workflows, research pipelines, or health‑tech products.
- **Leverage our synthetic data engine** to accelerate model training while safeguarding patient privacy.
- **Adopt our audit framework** to satisfy regulators and demonstrate responsible AI stewardship.

*By marrying cutting‑edge generative models with a rigorous ethical framework, we enable healthcare organizations to innovate safely, equitably, and transparently.*

## Task 3: Marketing Adverts for Smartphone

You have the technical fact sheets of one smartphone. Try some iterative prompt engineering and do the following:

1. Generate marketing product description for the smartphone

2. Custom product description which has the following:

```
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen,
processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put the most important features in the table which matter to users.
```

3. Custom product description focusing on specific aspects like display, camera and in less than 60 words

In [ ]:
# Define a multi-line string variable to hold the technical fact sheet for a mobile smartphone.
fact_sheet_mobile = """
PRODUCT NAME:
Samsung Galaxy Z Fold6
​
PRODUCT OVERVIEW:
Galaxy AI is Here - Put PC-like power in your pocket, Galaxy Z Fold6. More powerful than ever with its super-slim, productive screen. Now super-charged with Galaxy AI on foldables.
Fold open a mobile gaming beast with a massive screen made better with the Vision Booster's powerful brightness and clarity even in broad daylight.
Enjoy silky-smooth gaming with Vulkan, even in AAA games. Then, Snapdragon 8 Gen 3 for Galaxy renders graphics that are absolutely fire.
Meet Fold's most powerful camera system yet. Topped with an upgraded NPU, mind-blowing specs and ProVisual Engine, it'll transform your multimedia experience. Zoom way, way, way in while keeping noise down and resolution clear with ProVisual Engine. Its sharp analysis of the zoomed-in region enhances your shot dramatically.
With our high-resolution, 50MP camera, witness every detail come to life with stunning clarity and vibrancy.​

PRODUCT SPECS:
Network	Technology
GSM / CDMA / HSPA / EVDO / LTE / 5G
Launch	Announced	2024, July 10
Status	Available. Released 2024, July 24
Body	Dimensions	Unfolded: 153.5 x 132.6 x 5.6 mm
Folded: 153.5 x 68.1 x 12.1 mm
Weight	239 g (8.43 oz)
Build	Glass front (Gorilla Glass Victus 2) (folded), plastic front (unfolded), glass back (Gorilla Glass Victus 2), aluminum frame
SIM	Up to two Nano-SIM and multi eSIM
 	IP48 water resistant (up to 1.5m for 30 min)
Enhanced armor aluminum frame with tougher drop and scratch resistance (advertised)
Stylus support
Display	Type	Foldable Dynamic LTPO AMOLED 2X, 120Hz, HDR10+, 2600 nits (peak)
Size	7.6 inches, 185.2 cm2 (~91.0% screen-to-body ratio)
Resolution	1856 x 2160 pixels (~374 ppi density)
 	Cover display:
Dynamic LTPO AMOLED 2X, 120Hz, 2600 nits (peak), Corning Gorilla Glass Victus 2
6.3 inches, 968 x 2376 pixels, 410 ppi
Platform	OS	Android 14, up to 4 major Android upgrades, One UI 6.1.1
Chipset	Qualcomm SM8650-AC Snapdragon 8 Gen 3 (4 nm)
CPU	8-core (1x3.39GHz Cortex-X4 & 3x3.1GHz Cortex-A720 & 2x2.9GHz Cortex-A720 & 2x2.2GHz Cortex-A520)
GPU	Adreno 750 (1 GHz)
Memory	Card slot	No
Internal	256GB 12GB RAM, 512GB 12GB RAM, 1TB 12GB RAM
 	UFS 4.0
Main Camera	Triple	50 MP, f/1.8, 23mm (wide), 1.0µm, dual pixel PDAF, OIS
10 MP, f/2.4, 66mm (telephoto), 1.0µm, PDAF, OIS, 3x optical zoom
12 MP, f/2.2, 123˚, 12mm (ultrawide), 1.12µm
Features	LED flash, HDR, panorama
Video	8K@30fps, 4K@60fps, 1080p@60/120/240fps (gyro-EIS), 720p@960fps (gyro-EIS), HDR10+
Selfie camera	Single	4 MP, f/1.8, 26mm (wide), 2.0µm, under display
Cover camera:
10 MP, f/2.2, 24mm (wide), 1/3", 1.22µm
Features	HDR
Video	4K@30/60fps, 1080p@30/60fps, gyro-EIS
Sound	Loudspeaker	Yes, with stereo speakers
3.5mm jack	No
 	32-bit/384kHz audio
Tuned by AKG
Comms	WLAN	Wi-Fi 802.11 a/b/g/n/ac/6e, tri-band, Wi-Fi Direct
Bluetooth	5.3, A2DP, LE, aptX HD
Positioning	GPS, GALILEO, GLONASS, BDS, QZSS
NFC	Yes
Radio	No
USB	USB Type-C 3.2, OTG
Features	Sensors	Fingerprint (side-mounted), accelerometer, gyro, proximity, compass, barometer
 	Samsung DeX (desktop experience support)
Ultra Wideband (UWB) support
Circle to Search
Battery	Type	Li-Po 4400 mAh, non-removable
Charging	25W wired, QC2.0, 50% in 30 min (advertised)
15W wireless
4.5W reverse wireless
Misc	Colors	Navy, Silver Shadow, Pink, Black, White
Models	SM-F956B, SM-F956B/DS, SM-F956U, SM-F956U1, SM-F956N, SM-F956W, SM-F9560
SAR EU	1.24 W/kg (head)     1.39 W/kg (body)
Price	₹ 132,890 / $ 1,147.35 / £ 1,199.99 / € 1,245.00
Tests	Performance	AnTuTu: 1608744 (v10)
GeekBench: 6757 (v6)
3DMark: 4644 (Wild Life Extreme)
Display	1630 nits max brightness (measured)
Loudspeaker	-26.5 LUFS (Good)
Battery (new)
Active use score 11:31h
"""

In [ ]:
# Define the prompt for generating a brief marketing product description from a technical fact sheet.
# The AI is instructed to act as a marketing manager.
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.

Write a brief marketing product description.
Keep it organic and natural.
Do not use any artificial constructs or artifacts for example em dashes (—)

Technical specifications:
```{fact_sheet_mobile}```
"""

In [ ]:
# Print the constructed prompt to review its content before sending it to the LLM.
print(prompt)


Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.

Write a brief marketing product description.
Keep it organic and natural.
Do not use any artificial constructs or artifacts for example em dashes (—)

Technical specifications:
```
PRODUCT NAME:
Samsung Galaxy Z Fold6
​
PRODUCT OVERVIEW:
Galaxy AI is Here - Put PC-like power in your pocket, Galaxy Z Fold6. More powerful than ever with its super-slim, productive screen. Now super-charged with Galaxy AI on foldables.
Fold open a mobile gaming beast with a massive screen made better with the Vision Booster's powerful brightness and clarity even in broad daylight.
Enjoy silky-smooth gaming with Vulkan, even in AAA games. Then, Snapdragon 8 Gen 3 for Galaxy renders graphics that are absolutely fire.
Meet Fold's most powerful camera system yet. Topped with an upgraded NPU, 

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate the brief marketing product description.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated description in Markdown format.
display(Markdown(response))

Experience the future of mobile technology with the Samsung Galaxy Z Fold6. This foldable powerhouse combines a stunning 7.6-inch Dynamic AMOLED 2X display with PC-level performance, powered by the latest Snapdragon 8 Gen 3 chip. Whether you're gaming, multitasking, or capturing memories, the Galaxy Z Fold6 delivers exceptional brightness, smooth graphics, and a versatile triple camera system featuring a 50MP main sensor and advanced zoom capabilities. Built with durable Gorilla Glass Victus 2 and an enhanced aluminum frame, it’s designed to keep up with your active lifestyle. With IP48 water resistance, stylus support, and up to 1TB of storage, the Galaxy Z Fold6 is the ultimate device for productivity and entertainment on the go.

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate the brief marketing product description.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated description in Markdown format.
display(Markdown(response))

Discover the future of mobile with the Samsung Galaxy Z Fold6.  
Fold it open and you’re greeted by a 7.6‑inch Dynamic LTPO AMOLED 2X display that delivers 120 Hz smoothness, HDR10+ brilliance and a peak brightness of 2600 nits – perfect for gaming, streaming or working on the go.  

Under the hood, the Snapdragon 8 Gen 3 and 12 GB of RAM give you instant performance, while the 50‑MP triple camera system, powered by ProVisual Engine and an upgraded NPU, turns every shot into a masterpiece. Capture 8K video, zoom 3× optically and enjoy crystal‑clear selfies with the 4‑MP under‑display camera.  

The Galaxy AI and Vision Booster bring AI‑enhanced clarity to every scene, and the 4400 mAh battery with 25 W fast charging keeps you powered for a full day of multitasking. With a sleek aluminum frame, Gorilla Glass Victus 2 protection, stylus support and IP48 water resistance, the Z Fold6 is built to keep up with your lifestyle.  

Available now in navy, silver, pink, black and white – experience the next level of foldable technology.

In [ ]:
# Define the prompt for generating a custom product description with specific formatting.
# The AI is instructed to act as a marketing manager and focus on key features
# like foldable display, processing power, RAM, camera, and battery life,
# and include a table of specifications.
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.

Write a brief product description.

The description should follow this format:

<h3>Product Name<h3>

<Name of the smartphone>
​
<h3>Description<h3>

<Brief Overview of the features>
​
<h3>Product Specifications<h3>

<Table with key product feature specifications>
​
The description should only focus on the most important features
a customer might look for in a phone including the foldable display screen,
processing power, RAM, camera and battery life.

After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put the most important features in the table which matter to users.

Keep it organic and natural.
Do not use any artificial constructs or artifacts for example em dashes (—)

Technical specifications:
```{fact_sheet_mobile}```
"""

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate the custom product description with table.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated description in Markdown format.
display(Markdown(response))

<h3>Product Name</h3>

Samsung Galaxy Z Fold6

<h3>Description</h3>

Experience the future of mobile technology with the Samsung Galaxy Z Fold6. This foldable smartphone offers a stunning 7.6-inch Dynamic AMOLED 2X display that unfolds to deliver a large, immersive screen perfect for gaming, multitasking, and media consumption. Powered by the latest Snapdragon 8 Gen 3 processor and equipped with 12GB of RAM, it ensures smooth and responsive performance even during the most demanding tasks. Capture every moment in incredible detail with its advanced triple camera system featuring a 50MP main sensor and enhanced zoom capabilities. The 4400mAh battery supports fast wired and wireless charging, keeping you powered throughout the day. With its sleek design and cutting-edge features, the Galaxy Z Fold6 is built for productivity and entertainment on the go.

<h3>Product Specifications</h3>

| Feature           | Specification                  |
|-------------------|-------------------------------|
| Display           | 7.6 inches Foldable Dynamic AMOLED 2X, 120Hz, 2600 nits peak brightness |
| Processor         | Qualcomm Snapdragon 8 Gen 3 (8-core) |
| RAM               | 12GB                          |
| Storage Options   | 256GB / 512GB / 1TB UFS 4.0  |
| Main Camera       | Triple: 50MP (wide), 10MP (telephoto, 3x optical zoom), 12MP (ultrawide) |
| Battery           | 4400mAh, 25W wired fast charging, 15W wireless charging |
| Weight            | 239 g                         |

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate the custom product description with table.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated description in Markdown format.
display(Markdown(response))

<h3>Product Name</h3>
Samsung Galaxy Z Fold6

<h3>Description</h3>
The Galaxy Z Fold6 brings a truly immersive foldable experience with a 7.6‑inch Dynamic LTPO AMOLED 2X display that delivers 120 Hz refresh and a peak brightness of 2,600 nits—perfect for gaming, streaming, and everyday use. Powered by the Snapdragon 8 Gen 3 chipset and 12 GB of RAM, it handles multitasking and AAA titles with ease. Capture stunning moments with a 50 MP main camera, 3‑x optical zoom, and 8K video recording. A 4,400 mAh battery paired with 25 W wired fast charging (50 % in 30 min) keeps you powered all day. Built with Gorilla Glass Victus 2 and an aluminum frame, the Fold6 is both stylish and durable.

<h3>Product Specifications</h3>
<table>
  <tr>
    <th>Feature</th>
    <th>Specification</th>
  </tr>
  <tr>
    <td>Display Size</td>
    <td>7.6" Foldable Dynamic LTPO AMOLED 2X</td>
  </tr>
  <tr>
    <td>Resolution</td>
    <td>1856 × 2160 (374 ppi)</td>
  </tr>
  <tr>
    <td>Refresh Rate</td>
    <td>120 Hz</td>
  </tr>
  <tr>
    <td>Peak Brightness</td>
    <td>2,600 nits</td>
  </tr>
  <tr>
    <td>Processor</td>
    <td>Snapdragon 8 Gen 3 (4 nm)</td>
  </tr>
  <tr>
    <td>CPU Cores</td>
    <td>8‑core (1 × 3.39 GHz Cortex‑X4, 3 × 3.1 GHz Cortex‑A720, 2 × 2.9 GHz Cortex‑A720, 2 × 2.2 GHz Cortex‑A520)</td>
  </tr>
  <tr>
    <td>GPU</td>
    <td>Adreno 750</td>
  </tr>
  <tr>
    <td>RAM</td>
    <td>12 GB</td>
  </tr>
  <tr>
    <td>Storage Options</td>
    <td>256 GB / 512 GB / 1 TB (all with 12 GB RAM)</td>
  </tr>
  <tr>
    <td>Main Camera</td>
    <td>50 MP wide, 10 MP telephoto (3× optical zoom), 12 MP ultrawide</td>
  </tr>
  <tr>
    <td>Video Recording</td>
    <td>8K @ 30 fps, 4K @ 60 fps</td>
  </tr>
  <tr>
    <td>Battery</td>
    <td>4,400 mAh</td>
  </tr>
  <tr>
    <td>Charging</td>
    <td>25 W wired (50 % in 30 min), 15 W wireless</td>
  </tr>
  <tr>
    <td>Water Resistance</td>
    <td>IP48 (1.5 m for 30 min)</td>
  </tr>
  <tr>
    <td>Build</td>
    <td>Glass front (Gorilla Glass Victus 2), aluminum frame</td>
  </tr>
  <tr>
    <td>Price</td>
    <td>₹132,890 / $1,147.35</td>
  </tr>
</table>

In [ ]:
# Define the prompt for generating a catchy product description, at most 60 words,
# focusing on display and camera, and using emojis.
# The AI is instructed to act as a marketing manager.
prompt = f"""
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
in triple backticks.

Write a catchy product description which uses at most 60 words
and focuses on the most important things about the smartphone
which might matter to users like display and camera.
Use relevant emojis as needed.

Keep it organic and natural.
Do not use any artificial constructs or artifacts for example em dashes (—)

Technical specifications:
```{fact_sheet_mobile}```
"""

In [ ]:
# Call the get_completion function with the defined prompt and the 'gpt-4.1-mini' model
# to generate the catchy product description.
response = get_completion(prompt,
                          model='gpt-4.1-mini')
# Display the generated description in Markdown format.
display(Markdown(response))

Unfold brilliance with the Samsung Galaxy Z Fold6 🌟 Enjoy a massive 7.6" AMOLED display with 120Hz smoothness and 2600 nits brightness for stunning clarity. Capture every moment in vivid detail with the powerful 50MP triple camera 📸 and zoom like a pro. Fold, play, create—power meets innovation in your pocket! 🚀

In [ ]:
# Call the get_completion function with the defined prompt and the 'openai/gpt-oss-20b' model
# to generate the catchy product description.
response = get_completion(prompt,
                          model='openai/gpt-oss-20b')
# Display the generated description in Markdown format.
display(Markdown(response))

Unfold the future with the Samsung Galaxy Z Fold6: a 7.6‑inch Dynamic LTPO AMOLED 2X screen that glows 2600 nits bright, even in daylight. Capture every detail with a 50‑MP main camera, 3‑x optical zoom, and ProVisual Engine for crystal‑clear shots. 8K video, 12‑GB RAM, and 25‑W fast charge keep you powered. 📱✨📸

## Task 4: IT Support Resolution with Template Pattern

Ask the LLM to act as a customer support agent, process each customer ticket message and output the response in JSON with the following fields
```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Hint: Use the following prompt in a for loop

```
prompt = f"""
            Act as a customer support agent. For the IT support message mentioned below
            in triple backticks, create a response is JSON in the following format:

            {{
                orig_msg: The original customer message
                orig_lang: Detected language of the customer message e.g. Spanish
                category: 1-2 word describing the category of the problem
                trans_msg: Translated customer message in English
                response: Response to the customer in orig_lang
                trans_response: Response to the customer in English
            }}

            Message:
            '''{msg}'''
            """
```

In [ ]:
# Define a list of IT support messages in various languages, representing a support queue.
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.", # Portuguese: "I cannot sync my contacts with the phone. I always get a failure message."
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.", # Italian: "I have trouble printing documents remotely. The job is not sent to the network printer."
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。", # Japanese: "I replaced the printer toner, but the print quality has deteriorated. I need support."
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.", # Russian: "I cannot log into the time tracking system, an error message appears. I need help."
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?", # Turkish: "My internet connection is very slow and sometimes completely cuts out. Can you help me?"
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста." # Russian: "I cannot install the security update. An error code appears. Please help."
]

In [ ]:
# Initialize empty lists to store responses from GPT 4.1 mini and GPT OSS models
gpt4_responses = []
gptoss_responses = []

# Iterate through each message in the 'it_support_queue'
for msg in tqdm(it_support_queue):
    # Construct a prompt for the LLM to act as a customer support agent.
    # The prompt requests the response in a specific JSON format.
    prompt = f"""
                Act as a customer support agent. For the IT support message mentioned below
                in triple backticks, create a response in JSON in the following format:

                {{
                    orig_msg: The original customer message
                    orig_lang: Detected language of the customer message e.g. Spanish
                    category: 1-2 word describing the category of the problem
                    trans_msg: Translated customer message in English
                    response: Response to the customer in orig_lang
                    trans_response: Response to the customer in English
                }}

                Message:
                '''{msg}'''
                """
    # Get completion from the 'gpt-4.1-mini' model (OpenAI) and append to its response list
    response = get_completion(prompt,
                              model='gpt-4.1-mini')
    gpt4_responses.append(response)
    # Get completion from the 'openai/gpt-oss-20b' model and append to its response list
    response = get_completion(prompt,
                              model='openai/gpt-oss-20b')
    gptoss_responses.append(response)

100%|██████████| 6/6 [00:25<00:00,  4.31s/it]


In [ ]:
# Iterate through each response generated by the OpenAI model for IT support tickets.
for response in gpt4_responses:
  # Display the response in Markdown format.
  display(Markdown(response))
  print('-----')
  print('\n')

```json
{
  "orig_msg": "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
  "orig_lang": "Portuguese",
  "category": "Synchronization",
  "trans_msg": "I can't sync my contacts with the phone. I always receive a failure message.",
  "response": "Olá! Sentimos muito pelo inconveniente. Por favor, verifique se a conexão com a internet está estável e se as permissões de sincronização estão ativadas no seu telefone. Se o problema persistir, tente reiniciar o dispositivo ou atualizar o aplicativo de contatos. Estamos aqui para ajudar caso precise de mais assistência.",
  "trans_response": "Hello! We are sorry for the inconvenience. Please check if your internet connection is stable and if the sync permissions are enabled on your phone. If the problem persists, try restarting your device or updating the contacts app. We are here to help if you need further assistance."
}
```

-----




```json
{
  "orig_msg": "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
  "orig_lang": "Italian",
  "category": "Printing",
  "trans_msg": "I have problems printing documents remotely. The job is not sent to the network printer.",
  "response": "Ci dispiace per il problema riscontrato. Per favore, verifichi che la stampante di rete sia accesa e connessa correttamente. Inoltre, controlli che il suo dispositivo sia collegato alla stessa rete della stampante. Se il problema persiste, provi a riavviare sia il computer che la stampante. Restiamo a disposizione per ulteriori assistenza.",
  "trans_response": "We are sorry for the issue you are experiencing. Please check that the network printer is turned on and properly connected. Also, ensure that your device is connected to the same network as the printer. If the problem persists, try restarting both your computer and the printer. We are here to assist you further if needed."
}
```

-----




```json
{
  "orig_msg": "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
  "orig_lang": "Japanese",
  "category": "Printer",
  "trans_msg": "I replaced the printer toner, but the print quality has deteriorated. I need support.",
  "response": "トナー交換後に印刷品質が低下しているとのこと、ご不便をおかけして申し訳ありません。まず、トナーが正しく装着されているかご確認ください。また、プリンターのクリーニング機能をお試しいただくことをおすすめします。それでも問題が解決しない場合は、詳しい状況をお知らせください。サポートいたします。",
  "trans_response": "We apologize for the inconvenience caused by the deterioration in print quality after replacing the toner. Please first check if the toner is installed correctly. We also recommend trying the printer's cleaning function. If the problem persists, please provide more details so we can assist you further."
}
```

...

-----




```json
{
  "orig_msg": "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
  "orig_lang": "Turkish",
  "category": "Internet",
  "trans_msg": "My internet connection is very slow and sometimes completely cuts off. Can you help?",
  "response": "İnternet bağlantınızın yavaşlığı ve kesilme sorunuyla ilgileniyoruz. Lütfen modem ve yönlendiricinizi yeniden başlatmayı deneyin. Sorun devam ederse, lütfen bizimle tekrar iletişime geçin.",
  "trans_response": "We are addressing the slowness and disconnection issue with your internet connection. Please try restarting your modem and router. If the problem persists, please contact us again."
}
```

-----




```json
{
  "orig_msg": "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста.",
  "orig_lang": "Russian",
  "category": "Update Issue",
  "trans_msg": "I cannot install the security update. An error code appears. Please help.",
  "response": "Здравствуйте! Пожалуйста, сообщите, какой именно код ошибки появляется при установке обновления безопасности. Это поможет нам быстрее решить вашу проблему.",
  "trans_response": "Hello! Please let us know the exact error code that appears when installing the security update. This will help us resolve your issue more quickly."
}
```

-----




In [ ]:
# Iterate through each response generated by the Llama (Groq) model for IT support tickets.
for response in gptoss_responses:
  # Display the response in Markdown format.
  display(Markdown(response))
  print('-----')
  print('\n')

```json
{
  "orig_msg": "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
  "orig_lang": "Portuguese",
  "category": "Sync Issue",
  "trans_msg": "I can't sync my contacts with my phone. I always receive a failure message.",
  "response": "Olá! Vamos tentar resolver isso. Primeiro, verifique se o seu telefone está conectado à internet (Wi‑Fi ou dados móveis). Em seguida, abra o aplicativo de contatos, vá em Configurações > Sincronizar e certifique-se de que a opção de sincronização está ativada. Se o problema persistir, tente limpar o cache do aplicativo: Configurações > Aplicativos > Contatos > Armazenamento > Limpar Cache. Por fim, reinicie o telefone e tente sincronizar novamente. Se ainda não funcionar, informe-nos para que possamos investigar mais a fundo.",
  "trans_response": "Hello! Let's try to fix this. First, make sure your phone is connected to the internet (Wi‑Fi or mobile data). Then open the contacts app, go to Settings > Sync, and ensure the sync option is enabled. If the issue persists, try clearing the app cache: Settings > Apps > Contacts > Storage > Clear Cache. Finally, restart your phone and try syncing again. If it still doesn't work, let us know so we can investigate further."
}
```

-----




```json
{
  "orig_msg": "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
  "orig_lang": "Italian",
  "category": "Printing",
  "trans_msg": "I have problems printing documents remotely. The job is not sent to the network printer.",
  "response": "Mi dispiace per il problema. Ecco alcuni passaggi che può provare:\n1. Verificare che la stampante sia connessa alla rete e che il driver sia installato sul computer remoto.\n2. Controllare le impostazioni di condivisione della stampante sul server.\n3. Prova a stampare un documento di prova dalla stampante condivisa.\n4. Se utilizza una VPN, assicurarsi che il traffico verso la stampante sia consentito.\nSe il problema persiste, invii uno screenshot del messaggio di errore e il modello della stampante.",
  "trans_response": "I’m sorry for the inconvenience. Here are a few steps you can try:\n1. Make sure the printer is connected to the network and that the driver is installed on the remote computer.\n2. Check the printer sharing settings on the server.\n3. Try printing a test page from the shared printer.\n4. If you’re using a VPN, ensure that traffic to the printer is allowed.\nIf the issue persists, please send a screenshot of the error message and the printer model."
}
```

-----




```json
{
  "orig_msg": "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
  "orig_lang": "Japanese",
  "category": "Print Quality",
  "trans_msg": "I replaced the printer toner but the print quality has deteriorated. I need support.",
  "response": "ご連絡ありがとうございます。トナー交換後に印刷品質が低下したとのこと、以下の点をご確認ください。\n1. **トナーの正しい装着**：トナーが正しく挿入されているか、または密閉されているか確認してください。\n2. **プリンターヘッドのクリーニング**：ヘッドに汚れが付着していると印刷が不鮮明になります。プリンタのメニューからヘッドクリーニングを実行してください。\n3. **印刷設定の確認**：印刷品質設定が「標準」や「高速」になっていないか確認し、必要に応じて「高品質」に変更してください。\n4. **ドライバーの再インストール**：古いドライバーが原因の場合があります。メーカーサイトから最新ドライバーをダウンロードし、再インストールしてください。\n5. **別紙でのテスト印刷**：別の用紙でテスト印刷を行い、同様の問題が発生するか確認してください。\n上記を試しても改善しない場合は、弊社サポートセンター（電話：0120-XXX-XXX、メール：support@example.com）までご連絡ください。迅速に対応いたします。\n\nご不便をおかけし申し訳ございません。ご質問や追加のご要望がございましたら、お気軽にお知らせください。",
  "trans_response": "Thank you for contacting us. We understand that after replacing the toner, the print quality has deteriorated. Please check the following:\n1. **Correct toner installation**: Ensure the toner is properly seated and sealed.\n2. **Print head cleaning**: Dirt on the head can cause poor print quality. Use the printer’s menu to run a head cleaning.\n3. **Print settings**: Verify that the quality setting isn’t set to \"Standard\" or \"Fast\"; switch to \"High Quality\" if needed.\n4. **Reinstall drivers**: Outdated drivers can cause issues. Download the latest drivers from the manufacturer’s website and reinstall.\n5. **Test on a different sheet**: Print a test page on another sheet to see if the problem persists.\nIf the issue remains after trying these steps, please contact our support center (phone: 0120‑XXX‑XXX, email: support@example.com). We’ll resolve it promptly.\n\nWe apologize for the inconvenience. If you have any further questions or requests, feel free to let us know."
}
```

...

-----




```json
{
  "orig_msg": "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
  "orig_lang": "Turkish",
  "category": "Internet Speed",
  "trans_msg": "My internet connection is very slow and sometimes it cuts out completely. Can you help?",
  "response": "Merhaba, yaşadığınız yavaşlık ve kesinti sorununu çözmek için birkaç adım önerebilirim. Öncelikle modeminizin yeniden başlatılması, kabloların kontrol edilmesi ve Wi‑Fi sinyal gücünün artırılması gibi basit çözümleri deneyin. Eğer sorun devam ederse, internet servis sağlayıcınızla iletişime geçip bağlantı hızınızı ve sinyal kalitesini kontrol ettirin. Sorununuzu çözmek için buradayız, ek bilgi verirseniz daha spesifik yardım sağlayabilirim.",
  "trans_response": "Hello, I can suggest a few steps to resolve the slowdown and disconnection issue. First, try restarting your modem, checking cables, and boosting Wi‑Fi signal strength. If the problem persists, contact your internet service provider to verify your connection speed and signal quality. We're here to help; if you provide more details, we can offer more specific assistance."
}
```

-----




```json
{
  "orig_msg": "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста.",
  "orig_lang": "Russian",
  "category": "Update error",
  "trans_msg": "I cannot install the security update. An error code appears. Please help.",
  "response": "Здравствуйте! Попробуйте выполнить следующие шаги:\n1. Убедитесь, что у вас стабильное интернет‑соединение.\n2. Запустите средство устранения неполадок Windows Update: Пуск → Настройки → Обновление и безопасность → Устранение неполадок → Windows Update.\n3. Очистите кэш обновлений, удалив содержимое папки C:\\Windows\\SoftwareDistribution\\Download.\n4. Перезагрузите компьютер и снова запустите обновление.\nЕсли ошибка сохраняется, пришлите, пожалуйста, точный код ошибки, чтобы мы могли помочь более детально.",
  "trans_response": "Hello! Please try the following steps:\n1. Ensure you have a stable internet connection.\n2. Run the Windows Update Troubleshooter: Start → Settings → Update & Security → Troubleshoot → Windows Update.\n3. Clear the update cache by deleting the contents of C:\\Windows\\SoftwareDistribution\\Download.\n4. Restart your computer and try installing the update again.\nIf the error persists, please provide the exact error code so we can assist you further."
}
```

-----




In [ ]:
gpt4_responses[0]

'```json\n{\n  "orig_msg": "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",\n  "orig_lang": "Portuguese",\n  "category": "Synchronization",\n  "trans_msg": "I can\'t sync my contacts with the phone. I always receive a failure message.",\n  "response": "Olá! Sentimos muito pelo inconveniente. Por favor, verifique se a conexão com a internet está estável e se as permissões de sincronização estão ativadas no seu telefone. Se o problema persistir, tente reiniciar o dispositivo ou atualizar o aplicativo de contatos. Estamos aqui para ajudar caso precise de mais assistência.",\n  "trans_response": "Hello! We are sorry for the inconvenience. Please check if your internet connection is stable and if the sync permissions are enabled on your phone. If the problem persists, try restarting your device or updating the contacts app. We are here to help if you need further assistance."\n}\n```'

## Task 5: IT Support Resolution with Template Pattern using Structured Outputs (LangChain + Pydantic)

In this exercise, we'll refactor the previous IT support ticket processing solution to use **LangChain's structured outputs** with **Pydantic schemas**. This approach provides:
- Type-safe JSON outputs
- Automatic validation
- No manual JSON parsing required
- Cleaner, more maintainable code


### Key Advantages Over Previous Approach

1. **No JSON parsing errors** - Pydantic handles validation automatically
2. **Type safety** - Fields are properly typed (str, int, etc.)
3. **Better error messages** - Pydantic provides clear validation errors
4. **Direct object access** - Use `response.category` instead of `response['category']`
5. **Schema documentation** - Field descriptions are built into the model

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field

# Define Pydantic schema for structured output
class ITSupportResponse(BaseModel):
    """Structured response for IT support ticket handling."""

    orig_msg: str = Field(description="The original customer message")
    orig_lang: str = Field(description="Detected language of the customer message (e.g., Spanish, Portuguese, Japanese)")
    category: str = Field(description="1-2 word category describing the problem (e.g., Printer Issue, Login Error)")
    trans_msg: str = Field(description="Translated customer message in English")
    response: str = Field(description="Support response to the customer in their original language")
    trans_response: str = Field(description="Support response to the customer in English")

# Initialize the ChatOpenAI model with structured output
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

st_llm = llm.with_structured_output(ITSupportResponse)

# Initialize empty list to store responses
gpt4_structured_responses = []

# Iterate through each message in the 'it_support_queue'
for msg in tqdm(it_support_queue[:2]):

    prompt = [
        SystemMessage(content="""You are an expert multilingual IT customer support agent.
        Analyze the customer's message, detect the language, categorize the issue,
        translate it to English, and provide a helpful response in both the original
        language and English.

        Response format should be as per the given schema."""),
        HumanMessage(content=f"""Here is the Customer IT ticket message: {msg}""")
    ]

    response = st_llm.invoke(prompt)
    gpt4_structured_responses.append(response)

100%|██████████| 2/2 [00:06<00:00,  3.03s/it]


In [ ]:
gpt4_structured_responses[0]

ITSupportResponse(orig_msg='Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.', orig_lang='Portuguese', category='Sync Issue', trans_msg="I can't sync my contacts with the phone. I always receive a failure message.", response='Olá! Sentimos muito pelo inconveniente. Por favor, verifique se o seu telefone está conectado à internet e se as permissões de sincronização estão ativadas para o aplicativo de contatos. Tente também reiniciar o telefone e sincronizar novamente. Se o problema persistir, informe-nos para que possamos ajudar mais detalhadamente.', trans_response='Hello! We are sorry for the inconvenience. Please check if your phone is connected to the internet and if the sync permissions are enabled for the contacts app. Also, try restarting your phone and syncing again. If the problem persists, please let us know so we can assist you further.')

In [ ]:
gpt4_structured_responses[0].model_dump_json()

'{"orig_msg":"Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.","orig_lang":"Portuguese","category":"Sync Issue","trans_msg":"I can\'t sync my contacts with the phone. I always receive a failure message.","response":"Olá! Sentimos muito pelo inconveniente. Por favor, verifique se o seu telefone está conectado à internet e se as permissões de sincronização estão ativadas para o aplicativo de contatos. Tente também reiniciar o telefone e sincronizar novamente. Se o problema persistir, informe-nos para que possamos ajudar mais detalhadamente.","trans_response":"Hello! We are sorry for the inconvenience. Please check if your phone is connected to the internet and if the sync permissions are enabled for the contacts app. Also, try restarting your phone and syncing again. If the problem persists, please let us know so we can assist you further."}'

In [ ]:
gpt4_structured_responses[0].model_dump()

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'orig_lang': 'Portuguese',
 'category': 'Sync Issue',
 'trans_msg': "I can't sync my contacts with the phone. I always receive a failure message.",
 'response': 'Olá! Sentimos muito pelo inconveniente. Por favor, verifique se o seu telefone está conectado à internet e se as permissões de sincronização estão ativadas para o aplicativo de contatos. Tente também reiniciar o telefone e sincronizar novamente. Se o problema persistir, informe-nos para que possamos ajudar mais detalhadamente.',
 'trans_response': 'Hello! We are sorry for the inconvenience. Please check if your phone is connected to the internet and if the sync permissions are enabled for the contacts app. Also, try restarting your phone and syncing again. If the problem persists, please let us know so we can assist you further.'}

In [ ]:
processed_responses = []

for response in gpt4_structured_responses:
    processed_responses.append(response.model_dump())

processed_responses

[{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
  'orig_lang': 'Portuguese',
  'category': 'Sync Issue',
  'trans_msg': "I can't sync my contacts with the phone. I always receive a failure message.",
  'response': 'Olá! Sentimos muito pelo inconveniente. Por favor, verifique se o seu telefone está conectado à internet e se as permissões de sincronização estão ativadas para o aplicativo de contatos. Tente também reiniciar o telefone e sincronizar novamente. Se o problema persistir, informe-nos para que possamos ajudar mais detalhadamente.',
  'trans_response': 'Hello! We are sorry for the inconvenience. Please check if your phone is connected to the internet and if the sync permissions are enabled for the contacts app. Also, try restarting your phone and syncing again. If the problem persists, please let us know so we can assist you further.'},
 {'orig_msg': 'Ho problemi a stampare i documenti da remoto. Il lavoro non viene inv

In [ ]:
import pandas as pd

pd.DataFrame(processed_responses)

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can't sync my contacts with the phone. I alw...,Olá! Sentimos muito pelo inconveniente. Por fa...,Hello! We are sorry for the inconvenience. Ple...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printer Issue,I have problems printing documents remotely. T...,Ci dispiace per il problema riscontrato. Per f...,We are sorry for the issue you are experiencin...


## Task 6: LLM as a Classifier with Few Shot Prompting

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text without providing any examples.

Here we well focus on a news classification problem statement and apply the following techniques

- 1. Zero-shot prompting (no examples)
- 2. Few-shot prompting (with examples)



In [ ]:
# Sample news articles
news_articles = [
    """
    The Federal Reserve announced today a quarter-point interest rate hike,
    marking the third increase this year. Chair Jerome Powell stated that the
    decision reflects ongoing efforts to combat inflation, which remains above
    the central bank's 2% target. Markets responded with mixed reactions, with
    the S&P 500 closing down 0.8%. Economists predict further rate adjustments
    may be necessary in the coming months to stabilize the economy.
    """,

    """
    Scientists at MIT have developed a breakthrough artificial intelligence system
    that can predict protein structures with 95% accuracy. The new model, called
    ProFold-X, could revolutionize drug discovery and disease treatment. Lead
    researcher Dr. Sarah Chen explained that the system uses advanced deep learning
    techniques to analyze molecular patterns. The findings were published in Nature
    this week and have already attracted significant interest from pharmaceutical companies.
    """,

    """
    The local community came together yesterday for the annual charity marathon,
    raising over $150,000 for the children's hospital. More than 2,000 participants
    ran through downtown streets in perfect weather conditions. Mayor Johnson praised
    the organizers and volunteers for their dedication. The event featured live music,
    food stalls, and activities for families. All proceeds will go directly to funding
    new pediatric equipment and patient care programs.
    """
]

Zero-Shot Prompting

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text by providing no examples.
But we still maintain consistency of responses by giving it a list of potential news categories.


In [ ]:
# define news categories
categories = ["Business/Finance", "Technology/Science", "Politics", "Sports",
              "Entertainment", "Health", "Local/Community", "Other"]
str(categories)

"['Business/Finance', 'Technology/Science', 'Politics', 'Sports', 'Entertainment', 'Health', 'Local/Community', 'Other']"

In [ ]:
responses = []

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

for article in tqdm(news_articles):
    prompt = [
        SystemMessage(content=f"""You are a news classification expert.
        Classify the given news article into ONE of the following categories:
        {str(categories)}

        Respond with ONLY the category name, nothing else."""),

        HumanMessage(content=f"News Article:\n{article}")
    ]
    response = llm.invoke(prompt)
    responses.append(response.content)

100%|██████████| 3/3 [00:01<00:00,  2.35it/s]


In [ ]:
# Display zero-shot results
for article, category in zip(news_articles, responses):
    print(f"\nArticle: {article}")
    print(f"Category: {category}")
    print("-" * 80)


Article: 
    The Federal Reserve announced today a quarter-point interest rate hike,
    marking the third increase this year. Chair Jerome Powell stated that the
    decision reflects ongoing efforts to combat inflation, which remains above
    the central bank's 2% target. Markets responded with mixed reactions, with
    the S&P 500 closing down 0.8%. Economists predict further rate adjustments
    may be necessary in the coming months to stabilize the economy.
    
Category: Business/Finance
--------------------------------------------------------------------------------

Article: 
    Scientists at MIT have developed a breakthrough artificial intelligence system
    that can predict protein structures with 95% accuracy. The new model, called
    ProFold-X, could revolutionize drug discovery and disease treatment. Lead
    researcher Dr. Sarah Chen explained that the system uses advanced deep learning
    techniques to analyze molecular patterns. The findings were published in Nat

Few-shot Classification

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text by providing a few examples of inputs and outputs.

We also maintain consistency of responses by giving it a list of potential news categories.



In [ ]:
responses = []

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

for article in tqdm(news_articles):
    prompt = [
        SystemMessage(content=f"""You are a news classification expert.
        Classify news articles into ONE of the following categories:
        {str(categories)}

        Use these examples as reference for both classification and output format:

        Example 1:
        Article: "Apple Inc. reported record quarterly earnings of $95 billion,
        exceeding analyst expectations. The tech giant's iPhone sales surged 12%
        year-over-year, driven by strong demand in Asian markets."
        Output: 📊 Business/Finance

        Example 2:
        Article: "NASA's James Webb Space Telescope has captured stunning images
        of distant galaxies formed just 300 million years after the Big Bang.
        The discovery could reshape our understanding of early universe formation."
        Output: 🔬 Technology/Science

        Now classify the following article using the SAME output format."""),

        HumanMessage(content=f"News Article:\n{article}")
    ]

    response = llm.invoke(prompt)
    responses.append(response.content)

100%|██████████| 3/3 [00:01<00:00,  1.54it/s]


In [ ]:
# Display few-shot results
for article, category in zip(news_articles, responses):
    print(f"\nArticle: {article}")
    print(f"Classification: {category}")
    print("-" * 80)


Article: 
    The Federal Reserve announced today a quarter-point interest rate hike,
    marking the third increase this year. Chair Jerome Powell stated that the
    decision reflects ongoing efforts to combat inflation, which remains above
    the central bank's 2% target. Markets responded with mixed reactions, with
    the S&P 500 closing down 0.8%. Economists predict further rate adjustments
    may be necessary in the coming months to stabilize the economy.
    
Classification: 📊 Business/Finance
--------------------------------------------------------------------------------

Article: 
    Scientists at MIT have developed a breakthrough artificial intelligence system
    that can predict protein structures with 95% accuracy. The new model, called
    ProFold-X, could revolutionize drug discovery and disease treatment. Lead
    researcher Dr. Sarah Chen explained that the system uses advanced deep learning
    techniques to analyze molecular patterns. The findings were publishe

## Task 7: Advanced Prompting Patterns for Reasoning

Advanced prompting techniques can dramatically improve LLM performance on complex reasoning tasks. These patterns guide the model to think more carefully and produce more accurate results.

### Three Key Techniques:

#### 1. **Zero-Shot Chain of Thought (Zero-Shot CoT)**

**What it is:** Simply adding "Let's think step by step" to your prompt encourages the model to break down its reasoning process without providing any examples.

**When to use:**
- Medical diagnosis assistance
- Financial risk assessment
- Legal document analysis
- Complex decision-making tasks
- When you don't have example solutions

**Key insight:** This simple phrase activates the model's reasoning capabilities without requiring any training or examples.

---

#### 2. **Few-Shot Chain of Thought (Few-Shot CoT)**

**What it is:** Providing a few examples that demonstrate step-by-step reasoning, then asking the model to solve a new problem using the same reasoning pattern.

**When to use:**
- Domain-specific problems (medical, legal, financial)
- When you want consistent reasoning format
- Complex tasks that benefit from examples
- When zero-shot doesn't perform well enough

**Key insight:** Examples teach the model both the reasoning process AND the expected output format.

---

#### 3. **Self-Consistency**

**What it is:** Generate multiple reasoning paths with different temperatures (0.3, 0.6, 0.9), then use another LLM to consolidate the results and select the most consistent answer.

**When to use:**
- High-stakes medical decisions
- Financial investment analysis
- Legal case evaluation
- When you need confidence in the result
- Reducing hallucinations

**Key insight:** Multiple perspectives with different creativity levels, consolidated by a deterministic model, produce more reliable results.

---

#### Comparison Table

| Technique | Examples Needed | Multiple Runs | Best For |
|-----------|----------------|---------------|----------|
| Zero-Shot CoT | None | 1 | Quick medical triage, financial screening |
| Few-Shot CoT | 2-5 | 1 | Specialized diagnosis, legal analysis |
| Self-Consistency | None | 3 (different temps) | Critical healthcare/financial decisions |

---

Let's explore each technique with practical real-world examples!

In [ ]:
# Setup: Initialize models for reasoning tasks
from langchain_openai import ChatOpenAI

# We'll use GPT-4.1-mini for these examples
# Temperature=0 for consistent results in most examples
reasoning_model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

print("✅ Model initialized for reasoning experiments")

✅ Model initialized for reasoning experiments


#### Zero-shot Chain of Thought

In [ ]:
# Technique 1: Zero-Shot Chain of Thought

# Healthcare scenario: Patient symptom assessment
medical_case = """
A 45-year-old patient presents with the following symptoms:
- Persistent fatigue for 3 weeks
- Unexplained weight loss (10 lbs in 2 weeks)
- Increased thirst and frequent urination
- Blurred vision occasionally

What condition should be considered as a priority, and what immediate steps should be recommended?
"""

# WITHOUT Chain of Thought (baseline)
baseline_prompt = medical_case

print("❌ WITHOUT Chain of Thought:")
print("=" * 60)
response = reasoning_model.invoke(baseline_prompt)
display(Markdown(response.content))

❌ WITHOUT Chain of Thought:


The patient's symptoms—persistent fatigue, unexplained weight loss, increased thirst (polydipsia), frequent urination (polyuria), and occasional blurred vision—are classic signs suggestive of **new-onset diabetes mellitus**, particularly **type 1 diabetes** or possibly **type 2 diabetes with hyperglycemia**.

### Priority Condition to Consider:
**Diabetes mellitus**, with a concern for possible **hyperglycemia** or even **diabetic ketoacidosis (DKA)** if symptoms worsen.

### Rationale:
- **Increased thirst and frequent urination** are hallmark symptoms of hyperglycemia.
- **Weight loss** despite normal or increased appetite suggests catabolism due to insulin deficiency.
- **Fatigue** is common due to poor glucose utilization.
- **Blurred vision** can result from osmotic changes in the lens due to high blood sugar.

### Immediate Steps to Recommend:
1. **Urgent blood glucose testing:**
   - Perform a **random blood glucose** or **fasting blood glucose** test immediately.
   - If available, check **point-of-care glucose** in the clinic or emergency setting.

2. **Assess for diabetic ketoacidosis (DKA):**
   - Check for symptoms such as nausea, vomiting, abdominal pain, rapid breathing.
   - Obtain blood gas analysis, serum ketones, and electrolytes if DKA is suspected.

3. **Referral and further testing:**
   - If hyperglycemia is confirmed, order HbA1c to assess chronic glucose control.
   - Urinalysis for glucose and ketones.
   - Consider referral to endocrinology or admission if DKA or severe hyperglycemia is present.

4. **Patient education:**
   - Advise the patient to seek emergency care if symptoms worsen (e.g., vomiting, abdominal pain, confusion).
   - Discuss the importance of hydration.

### Summary:
- **Priority diagnosis:** New-onset diabetes mellitus (likely type 1 or type 2).
- **Immediate action:** Check blood glucose urgently, evaluate for DKA, and initiate appropriate management or referral.

If you want, I can also provide guidance on initial management steps after diagnosis.

In [ ]:
# WITH Zero-Shot Chain of Thought
# Key: Adding "Let's think step by step" triggers systematic reasoning
zero_shot_cot_prompt = f"""
{medical_case}

Think step by step to analyze these symptoms systematically.
Then generate the detailed final response.
"""

print("\n✅ WITH Zero-Shot Chain of Thought:")
print("=" * 60)
response = reasoning_model.invoke(zero_shot_cot_prompt)
display(Markdown(response.content))


✅ WITH Zero-Shot Chain of Thought:


Let's analyze the patient's symptoms step-by-step:

### Step 1: List the symptoms
- Persistent fatigue for 3 weeks
- Unexplained weight loss (10 lbs in 2 weeks)
- Increased thirst (polydipsia)
- Frequent urination (polyuria)
- Occasional blurred vision

### Step 2: Consider possible causes for each symptom
- **Fatigue**: Can be caused by many conditions including infections, anemia, metabolic disorders, endocrine disorders (e.g., hypothyroidism, diabetes), depression, malignancy.
- **Unexplained weight loss**: Suggests a catabolic state or increased metabolism; common causes include malignancy, hyperthyroidism, diabetes mellitus, chronic infections.
- **Increased thirst and frequent urination**: Classic symptoms of hyperglycemia (high blood sugar), diabetes insipidus, or other renal concentrating defects.
- **Blurred vision**: Can be caused by hyperglycemia (osmotic changes in the lens), hypertension, or other ocular conditions.

### Step 3: Look for a unifying diagnosis
- The combination of **increased thirst, frequent urination, weight loss, and fatigue** strongly suggests **diabetes mellitus**, particularly new-onset or poorly controlled diabetes.
- Blurred vision can be explained by hyperglycemia causing changes in the lens and refractive index.
- The rapid weight loss and fatigue are consistent with uncontrolled diabetes leading to catabolism of fat and muscle.

### Step 4: Consider urgency and complications
- These symptoms could indicate **new-onset diabetes mellitus**, possibly progressing to **diabetic ketoacidosis (DKA)**, especially if the patient has nausea, vomiting, abdominal pain, or altered mental status (not mentioned here but important to ask).
- DKA is a medical emergency.
- Even if not in DKA, hyperglycemia needs urgent diagnosis and management to prevent complications.

### Step 5: Immediate steps to recommend
1. **Obtain urgent blood tests:**
   - Random blood glucose or fasting blood glucose
   - HbA1c (to assess chronic glucose control)
   - Serum electrolytes, blood urea nitrogen (BUN), creatinine (to assess kidney function)
   - Serum ketones or beta-hydroxybutyrate (to rule out DKA)
   - Arterial blood gas if DKA suspected
2. **Urinalysis:**
   - Check for glucose and ketones in urine
3. **Physical examination:**
   - Vital signs (check for signs of dehydration, tachycardia, hypotension)
   - Look for signs of infection or other causes
4. **If hyperglycemia confirmed:**
   - Refer to emergency or endocrinology for management
   - Initiate fluid resuscitation if dehydrated
   - Start insulin therapy if indicated
5. **Patient education:**
   - Advise on symptoms of DKA and when to seek emergency care
   - Discuss lifestyle modifications and follow-up

---

## Final Summary and Recommendation:

**Priority condition to consider:**  
**New-onset diabetes mellitus**, with a high suspicion for hyperglycemia and possible impending diabetic ketoacidosis.

**Immediate recommended steps:**  
- Perform urgent blood glucose testing (random or fasting) and check for ketones.  
- Conduct urinalysis for glucose and ketones.  
- Assess hydration status and vital signs.  
- If hyperglycemia and/or ketones are present, initiate prompt referral to emergency care for further evaluation and management, including possible insulin therapy and fluid replacement.  
- Educate the patient about the seriousness of symptoms and the need for urgent medical attention if symptoms worsen.

---

If you want, I can also provide a differential diagnosis or further management details.

#### Few-Shot Chain of Thought

In [ ]:
# Financial investment risk assessment
# New investment scenario to analyze
new_investment = """
Investment Scenario:
Company: TechStart AI Inc. (Tech Startup)
- Revenue: $2M (grew 300% last year)
- Burn rate: $500K/month
- Cash reserves: $4M
- Industry: AI/Machine Learning
- Competition: High (5 major competitors)
- Market: Growing 40% annually
- Valuation: $50M (Series A)

Should an investor proceed with a $1M investment?
"""

# Few-Shot CoT: Show examples of investment analysis reasoning
few_shot_cot_prompt = f"""
Here are examples of systematic investment analysis:

Example 1:
Investment: E-commerce Startup XYZ
- Revenue: $5M (grew 50% last year)
- Burn rate: $200K/month
- Cash reserves: $3M
- Industry: E-commerce fashion
- Competition: Moderate
- Market: Growing 15% annually
- Valuation: $30M

Analysis Steps:
1. Runway calculation: $3M / $200K = 15 months runway
2. Revenue growth: 50% YoY is solid but not exceptional
3. Market analysis: 15% growth is stable but not explosive
4. Competition: Moderate means market share is achievable
5. Valuation check: $30M on $5M revenue = 6x multiple (reasonable)
6. Risk assessment: Moderate risk, decent runway, proven model

Recommendation: PROCEED with caution. Good fundamentals but ensure milestones are clear.

Example 2:
Investment: BioTech Pharma Co.
- Revenue: $500K (grew 20% last year)
- Burn rate: $1M/month
- Cash reserves: $8M
- Industry: Pharmaceutical R&D
- Competition: Low (novel drug)
- Market: Growing 25% annually
- Valuation: $100M (Series B)

Analysis Steps:
1. Runway calculation: $8M / $1M = 8 months runway (concerning!)
2. Revenue growth: 20% growth with low revenue is risky
3. Market analysis: 25% growth is good, pharma has potential
4. Competition: Low competition is positive for novel drug
5. Valuation check: $100M on $500K revenue = 200x (very high!)
6. Risk assessment: HIGH risk - short runway, high burn, early stage

Recommendation: DO NOT PROCEED unless they secure additional funding first. Runway too short.


Now analyze this new investment opportunity using the same systematic approach:

{new_investment}
"""

print("Few-Shot Chain of Thought - Investment Analysis:")
print("=" * 60)
response = reasoning_model.invoke(few_shot_cot_prompt)
display(Markdown(response.content))

Few-Shot Chain of Thought - Investment Analysis:


Let's analyze TechStart AI Inc. using the same systematic approach:

---

### Investment: TechStart AI Inc.

- **Revenue:** $2M (grew 300% last year)
- **Burn rate:** $500K/month
- **Cash reserves:** $4M
- **Industry:** AI/Machine Learning
- **Competition:** High (5 major competitors)
- **Market:** Growing 40% annually
- **Valuation:** $50M (Series A)

---

### Analysis Steps:

1. **Runway calculation:**  
   Runway = Cash reserves / Burn rate = $4M / $0.5M = 8 months runway  
   - This is relatively short and could be concerning unless they raise more funds soon or reduce burn.

2. **Revenue growth:**  
   300% YoY growth is exceptional and indicates strong traction and product-market fit.

3. **Market analysis:**  
   40% annual growth in AI/ML is very strong, indicating a large and expanding opportunity.

4. **Competition:**  
   High competition with 5 major competitors means the company must have a strong differentiator or risk losing market share.

5. **Valuation check:**  
   Valuation / Revenue = $50M / $2M = 25x multiple  
   - This is high compared to traditional industries but not unusual for high-growth AI startups at Series A.

6. **Risk assessment:**  
   - **Positives:** Exceptional revenue growth, large and fast-growing market.  
   - **Negatives:** Short runway (8 months), high competition, high valuation multiple.  
   - Burn rate is significant relative to revenue, so cash management and next funding round are critical.

---

### Recommendation:

**Proceed with caution, conditional on:**

- Clear milestones and path to profitability or next funding round within 8 months.  
- Strong competitive advantages or IP to defend market share.  
- Investor should negotiate terms that protect downside (e.g., pro-rata rights, board seat).  
- Monitor burn rate closely and ensure management has a plan to extend runway.

---

### Summary:

- The company shows **strong growth and market potential** but has a **short runway and high competition**.  
- The **valuation is high but justifiable** given growth and sector.  
- A $1M investment could be worthwhile **if the investor is comfortable with risk and the company has a clear plan to manage cash and competition.**

---

**Final verdict:**  
**PROCEED with caution and strong due diligence.**

#### Self-Consistency

In [ ]:
# Legal contract clause interpretation
# Strategy: Run with 3 different LLMs, then consolidate
# Legal scenario: Contract dispute analysis
legal_case = """
Contract Clause Analysis:

Clause: "The Supplier shall deliver goods within 30 business days of order confirmation,
subject to force majeure events. In case of delay, Buyer may cancel the order without
penalty if delivery exceeds 45 business days from confirmation."

Scenario:
- Order confirmed: January 15, 2024
- Severe snowstorm (declared state of emergency): February 1-10, 2024
- Delivery date: March 15, 2024
- Business days elapsed: 43 days (excluding 7 snow days)
- Buyer wants to cancel and claims breach of contract

Question: Can the Buyer legally cancel the order and claim breach of contract?
"""

# Initialize models with different temperatures
# Low temp (0.3)
# Medium temp (0.6)
# High temp (0.9)

llm1 = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
llm2 = ChatOpenAI(model="gpt-4.1-mini", temperature=0.6)
llm3 = ChatOpenAI(model="gpt-4.1-mini", temperature=0.9)

# Consolidator model (temperature=0 for deterministic output)
llm_consolidator = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

print("Self-Consistency: Legal Contract Analysis")
print("=" * 70)
print(f"\n{legal_case}\n")
print("=" * 70)

Self-Consistency: Legal Contract Analysis


Contract Clause Analysis:

Clause: "The Supplier shall deliver goods within 30 business days of order confirmation,
subject to force majeure events. In case of delay, Buyer may cancel the order without
penalty if delivery exceeds 45 business days from confirmation."

Scenario:
- Order confirmed: January 15, 2024
- Severe snowstorm (declared state of emergency): February 1-10, 2024
- Delivery date: March 15, 2024
- Business days elapsed: 43 days (excluding 7 snow days)
- Buyer wants to cancel and claims breach of contract

Question: Can the Buyer legally cancel the order and claim breach of contract?




In [ ]:
# Step 1: Generate 3 responses with different temperatures

prompt = f"""
{legal_case}

Analyze this legal scenario step by step and provide your conclusion.
"""

# Response 1: LLM (temp=0.3)
print("\n🔷 PERSPECTIVE 1: LLM (Temperature=0.3)")
print("-" * 70)
response_1 =llm1.invoke(prompt)
display(Markdown(response_1.content))

# Response 2: LLM (temp=0.6)
print("\n\n🔶 PERSPECTIVE 2: LLM (Temperature=0.6)")
print("-" * 70)
response_2 = llm2.invoke(prompt)
display(Markdown(response_2.content))

# Response 3: LLM (temp=0.9)
print("\n\n🔸 PERSPECTIVE 3: LLM (Temperature=0.9)")
print("-" * 70)
response_3 = llm3.invoke(prompt)
display(Markdown(response_3.content))

# Store responses for consolidation
responses_to_consolidate = [
    response_1.content,
    response_2.content,
    response_3.content
]

print("\n" + "=" * 70)
print("✅ Generated 3 different perspectives. Now consolidating...\n")


🔷 PERSPECTIVE 1: LLM (Temperature=0.3)
----------------------------------------------------------------------


Let's analyze the contract clause and the scenario step by step:

---

### Contract Clause Recap:
1. **Delivery timeline:** Supplier must deliver goods within **30 business days** of order confirmation.
2. **Force majeure exception:** Delivery timeline is **subject to force majeure events**.
3. **Cancellation right:** Buyer may cancel **without penalty** if delivery exceeds **45 business days** from confirmation.

---

### Scenario Facts:
- **Order confirmation date:** January 15, 2024
- **Force majeure event:** Severe snowstorm, declared state of emergency, from February 1 to February 10, 2024 (10 calendar days)
- **Delivery date:** March 15, 2024
- **Business days elapsed:** 43 days (excluding 7 snow days)
- **Buyer’s position:** Wants to cancel and claims breach of contract.

---

### Step 1: Calculate the delivery deadline without force majeure

- Starting point: January 15, 2024
- Add 30 business days (excluding weekends and holidays)
- Assume standard US business days (Monday to Friday)
- 30 business days from January 15 would roughly be around **February 26, 2024** (approximate, but close enough for analysis).

---

### Step 2: Consider the force majeure event

- The clause states delivery timeline is **subject to force majeure events**.
- The snowstorm was a declared state of emergency from **February 1 to February 10**.
- The snowstorm days should be **excluded** from the business day count because they are force majeure days.
- The Buyer excludes 7 snow days from the business day count, which seems reasonable (some of the 10 days include weekends).

---

### Step 3: Calculate the extended delivery deadline including force majeure

- Original deadline: ~February 26, 2024 (30 business days)
- Add the 7 force majeure business days (snowstorm days)
- New deadline: approximately **March 6, 2024**

---

### Step 4: Check if delivery was late beyond cancellation threshold

- Buyer can cancel if delivery exceeds **45 business days** from confirmation.
- 45 business days from January 15, excluding 7 force majeure days, would be:
  - 45 business days + 7 force majeure days = 52 calendar business days effectively.
- Counting 45 business days from January 15 excluding force majeure days would land roughly around **March 18, 2024**.

- Actual delivery date: March 15, 2024
- Business days elapsed (excluding snow days): 43 days

---

### Step 5: Determine if Buyer can cancel

- Delivery was made **within 45 business days** (43 business days elapsed).
- Delivery was **after 30 business days**, but force majeure days extend the deadline.
- Buyer’s right to cancel triggers only if delivery exceeds 45 business days.
- Since delivery was made before 45 business days, Buyer **cannot cancel without penalty**.

---

### Step 6: Is there a breach of contract?

- Supplier delivered late relative to the original 30 business days.
- However, the delay is excused due to force majeure.
- Delivery was made before the cancellation threshold.
- Therefore, **no breach of contract** occurred.

---

## **Conclusion:**

- The Buyer **cannot legally cancel the order without penalty** because the delivery was made within the extended deadline accounting for force majeure.
- The Supplier is **not in breach of contract** as the delay was excused by the force majeure clause.
- Buyer’s claim to cancel and claim breach is **not supported** under the contract terms.

---

If you want, I can also draft a formal response or legal memo based on this analysis.



🔶 PERSPECTIVE 2: LLM (Temperature=0.6)
----------------------------------------------------------------------


Let's analyze the scenario step-by-step based on the contract clause and facts provided.

---

### Contract Clause Summary:
- **Delivery timeframe:** 30 business days from order confirmation.
- **Force majeure:** Delivery timeframe is subject to force majeure events.
- **Delay and cancellation:** If delivery exceeds 45 business days, Buyer may cancel without penalty.

---

### Facts:
- **Order confirmed:** January 15, 2024
- **Force majeure event:** Severe snowstorm declared state of emergency from February 1 to February 10, 2024 (10 calendar days)
- **Delivery date:** March 15, 2024
- **Business days elapsed:** 43 business days excluding 7 snow days
- **Buyer wants to cancel** and claims breach.

---

### Step 1: Understand the delivery deadline

- **Initial delivery deadline:** 30 business days from Jan 15, 2024.
- Count 30 business days starting Jan 15, 2024, **excluding** weekends and holidays.

---

### Step 2: Impact of force majeure

- The clause says delivery timeframe is **subject to force majeure events**.
- The snowstorm was declared a state of emergency, qualifying as a force majeure event.
- Force majeure suspends or extends the delivery timeline for the duration of the event.
- The snowstorm lasted from Feb 1 to Feb 10, 2024 (10 calendar days).
- However, the Buyer excludes 7 snow days from business days count, implying 7 business days during the snowstorm were non-working.

---

### Step 3: Calculate adjusted delivery deadline

- Original deadline: 30 business days from Jan 15.
- Add 7 business days of force majeure suspension to the deadline.
- Therefore, new delivery deadline = 30 + 7 = 37 business days from Jan 15.

---

### Step 4: Check if delivery exceeded cancellation threshold

- Buyer can cancel if delivery exceeds 45 business days.
- The actual delivery took 43 business days **excluding** 7 snow days.
- Adding 7 snow days gives a total of 50 business days if snow days are counted.
- But since force majeure suspends the timeline, the 7 snow days **should not** count towards the business days elapsed.
- So, delivery was effectively made on day 43 + 7 suspended days = day 50 since order confirmation, but only 43 counted business days elapsed.

---

### Step 5: Interpretation of "exceeds 45 business days"

- The clause likely means **45 elapsed business days excluding force majeure suspended days**.
- Since the delivery took 43 counted business days (excluding force majeure days), delivery did **not** exceed 45 business days.
- The Buyer cannot count force majeure days toward the 45 business days threshold.

---

### Step 6: Conclusion

- Delivery was late beyond the original 30 business days deadline.
- However, the delay was due to force majeure, which suspends the delivery timeline.
- The adjusted delivery timeframe (30 business days + 7 force majeure days) was **not exceeded**.
- Delivery was made within 43 business days (excluding force majeure days), which is within the 45 business days cancellation threshold.
- Therefore, Buyer **cannot legally cancel the order** without penalty nor claim breach of contract.

---

### Final conclusion:

**The Buyer cannot legally cancel the order or claim breach of contract because the delivery delay was excused by the force majeure event, and the delivery was made within the extended timeframe allowed by the contract.**



🔸 PERSPECTIVE 3: LLM (Temperature=0.9)
----------------------------------------------------------------------


Let's analyze the scenario step-by-step, focusing on the contract clause language, the timeline, and the impact of the force majeure event.

---

### 1. **Contract Clause Summary**
- **Delivery timeframe:** Supplier must deliver within 30 business days of order confirmation.
- **Force majeure:** Delivery deadline is **subject to force majeure events**.
- **Delay remedy:** Buyer can cancel without penalty if delivery is **beyond 45 business days** from confirmation.

---

### 2. **Timeline of Events**

| Date             | Event                                | Notes                          |
|------------------|------------------------------------|--------------------------------|
| Jan 15, 2024     | Order confirmed                    | Day 0                          |
| Feb 1-10, 2024   | Severe snowstorm declared           | State of emergency, force majeure period |
| Mar 15, 2024     | Delivery occurred                   | Actual delivery date            |

---

### 3. **Key Points to Consider**

#### A. **Calculation of Business Days**

- **Business days elapsed:** 43 days excluding snowstorm days.
- The clause excludes force majeure days from the delivery timeframe, so days from Feb 1-10 are not counted as regular business days.

Assuming:

- Snowstorm days (Feb 1-10) are considered force majeure days.
- Therefore, these 7 business days are excluded from the count.

---

#### B. **Effect of Force Majeure**

- Clause explicitly makes the 30-business-day delivery timeframe **subject to force majeure events**.
- This typically means:
  - The Supplier’s delivery obligation is suspended or extended during force majeure.
  - The days of force majeure (Feb 1-10) are **excluded** from the delivery calculation.
  
This means the 30-business-day window should be **extended by the length of the force majeure period** (7 days).

---

#### C. **When Did the Delivery Deadline Actually Elapse?**

- Normal delivery deadline: 30 business days from Jan 15.
- Add 7-day force majeure extension.
- Effective delivery deadline = 37 business days from Jan 15 (counting only non-force majeure days).

- Buyer can cancel only after **45 business days** post-confirmation (from Jan 15), as per clause.

- Supplier delivered on day 43 (excluding force majeure days).

- Thus, actual delivery was **within the 45-business-day cancellation threshold**.

---

### 4. **Buyer's Right to Cancel**

- Buyer may cancel **only if delivery exceeds 45 business days**.

- Delivery occurred at 43 business days (excluding force majeure), which is **less than 45 days**.

- Therefore, the Buyer **does not have the right to cancel without penalty** and cannot claim breach of contract on this basis.

---

### 5. **Additional Considerations**

- **Force majeure declaration:** Since a state of emergency was declared during Feb 1-10, it supports that this was a valid force majeure event.
- **Any contractual notice or procedural requirements for cancellation?** Not specified here, but Buyer must comply with all contract terms.
- **Buyer claiming breach:** Since Supplier performed within extended deadline, there is no breach based on delay.

---

### **Conclusion**

- Because the clause extends the delivery deadline by the force majeure period, and delivery was made **within 43 business days excluding the 7 force majeure days (thus within 30 + 7 = 37 business days delivery window and less than the 45-day cancellation threshold)**,
- **Buyer cannot legally cancel the order due to delay, nor claim breach of contract.**

---

**Final Legal Position:** The Buyer’s cancellation is not supported by the contract terms, and the Supplier has fulfilled its delivery obligation within the permissible timeframe when accounting for the force majeure event.


✅ Generated 3 different perspectives. Now consolidating...



In [ ]:
# Step 2: Consolidate all responses using a deterministic model

consolidation_prompt = f"""
You are a senior legal analyst tasked with reviewing multiple legal analyses and providing a final, consolidated opinion.

Below are THREE different analyses of the same legal scenario. Your job is to:
1. Review all three perspectives
2. Identify common conclusions and reasoning
3. Note any significant disagreements
4. Apply majority voting where applicable
5. Provide a FINAL CONSOLIDATED RECOMMENDATION

Original Legal Scenario:
{legal_case}

---

ANALYSIS 1:
{responses_to_consolidate[0]}

---

ANALYSIS 2:
{responses_to_consolidate[1]}

---

ANALYSIS 3:
{responses_to_consolidate[2]}

---

Based on these three analyses, provide your CONSOLIDATED FINAL RECOMMENDATION:
- What is the majority conclusion?
- What are the strongest legal arguments?
- What is your final answer to: "Can the Buyer legally cancel the order and claim breach of contract?"

Format your response as:
1. Majority Conclusion
2. Key Supporting Arguments
3. Final Recommendation
"""

print("🎯 CONSOLIDATED ANALYSIS:")
print("=" * 70)
final_response = llm_consolidator.invoke(consolidation_prompt)
display(Markdown(final_response.content))

print("\n" + "=" * 70)
print("✅ Self-Consistency Complete: Final recommendation generated!")

🎯 CONSOLIDATED ANALYSIS:


**1. Majority Conclusion**

All three analyses reach the same conclusion:  
**The Buyer cannot legally cancel the order without penalty and cannot claim breach of contract.** The Supplier’s delivery, although delayed beyond the original 30 business days, was made within the extended timeframe allowed by the force majeure clause and before the 45-business-day cancellation threshold.

---

**2. Key Supporting Arguments**

- **Force Majeure Clause Suspension:** The contract explicitly states that the delivery timeline is subject to force majeure events. The declared state of emergency due to the severe snowstorm qualifies as a force majeure event, suspending or extending the delivery deadline.

- **Exclusion of Force Majeure Days:** The 7 business days during the snowstorm (Feb 1-10) are excluded from the business day count, effectively extending the delivery deadline by those days.

- **Adjusted Delivery Deadline:**  
  - Original delivery deadline: 30 business days from January 15 (~Feb 26).  
  - Adding 7 force majeure days extends this to approximately 37 business days (~March 6).

- **Cancellation Threshold:** Buyer may cancel only if delivery exceeds 45 business days (excluding force majeure days). The actual delivery occurred on the 43rd business day (excluding force majeure days), which is within the cancellation threshold.

- **No Breach of Contract:** Since the Supplier delivered within the extended deadline and before the cancellation threshold, the delay is excused, and no breach has occurred.

---

**3. Final Recommendation**

**The Buyer does not have a contractual right to cancel the order without penalty nor to claim breach of contract based on the delivery delay.** The Supplier’s delivery on March 15, 2024, complies with the contract terms when accounting for the force majeure event. The Buyer’s cancellation demand and breach claim should be rejected.

---

If needed, a formal legal memo or response can be drafted to communicate this position to the Buyer or relevant parties.


✅ Self-Consistency Complete: Final recommendation generated!


---

### Summary: Advanced Prompting Techniques - Real-World Applications

#### ✅ Zero-Shot Chain of Thought

**Real-world uses:**
- Initial medical symptom screening
- Quick financial health check
- Preliminary legal document review
- Customer support ticket triage

**Example:** "Patient has fever and cough. Let's think step by step about possible causes and urgency level."

**Pros:** Fast, no setup, surprisingly accurate  
**Cons:** May miss domain nuances

---

#### ✅ Few-Shot Chain of Thought

**Real-world uses:**
- Specialized medical diagnosis (radiology, pathology)
- Investment portfolio analysis
- Complex contract interpretation
- Insurance claim assessment

**Example:** Show 2-3 examples of how to analyze financial statements, then ask model to analyze a new company.

**Pros:** Domain-specific expertise, consistent format  
**Cons:** Requires quality examples, longer prompts

---

#### ✅ Self-Consistency (Multi-Temperature Consolidation)

**Real-world uses:**
- Critical insurance decisions
- Large investment approvals (>$1M)
- High-stakes legal cases
- Drug interaction checking

**Process:**
1. Run with temp 0.3
2. Run with temp 0.6
3. Run with temp 0.9
4. Consolidate with temp 0 (deterministic)

**Pros:** Highest accuracy, catches edge cases, reduces errors  
**Cons:** 4x API calls, slower, more expensive

---

### Decision Matrix - Examples

| Scenario | Recommended Technique |
|----------|----------------------|
| ER triage | Zero-Shot CoT |
| Investment screening (<\$100K) | Zero-Shot CoT |
| Specialized diagnosis | Few-Shot CoT |
| Portfolio optimization | Few-Shot CoT |
| Insurance decision | Self-Consistency |
| Major investment (>\$1M) | Self-Consistency |

---

### Cost vs Accuracy vs Speed

**Zero-Shot CoT:**
- Cost: \$ (1 call)
- Speed: ⚡⚡⚡ Very Fast
- Accuracy: ⭐⭐⭐ Good

**Few-Shot CoT:**
- Cost: \$ (1 call)
- Speed: ⚡⚡ Fast
- Accuracy: ⭐⭐⭐⭐ Very Good

**Self-Consistency:**
- Cost: \$$$$ (4 calls)
- Speed: ⚡ Slower
- Accuracy: ⭐⭐⭐⭐⭐ Excellent

---

### 💡 Pro Tips

#### Combine techniques for critical decisions:
**Few-Shot CoT + Self-Consistency = Best results**


#### Always validate with domain experts:
- **Medical decisions** → Board-certified physicians
- **Legal advice** → Licensed attorneys
- **Financial advice** → CFAs/Financial advisors

---

## Next Steps

Now that you understand advanced prompting patterns, you can:

- Apply these techniques to your specific domain
- Experiment with different temperature combinations
- Build evaluation frameworks to measure accuracy
- Combine with RAG (Retrieval Augmented Generation) and AI Agents for even better results